<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/Neuron/blob/main/Tutorial_12_Learning_from_Experience_Adaptive_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In Tutorial 11, you built sophisticated deliberative reasoning systems.
Now we're adding comprehensive learning capabilities - agents that improve  their performance through experience, adapt to new situations, and develop expertise over time.

 What you'll learn:

 • Experience-based learning and adaptation mechanisms

 • Pattern recognition and knowledge extraction from interactions

 • Multi-modal learning (supervised, unsupervised, reinforcement)

 • Expertise development and specialization

 • Knowledge transfer between agents

 • Continuous improvement and performance optimization

Why this matters:

True intelligence emerges from the ability to learn and adapt. Agents that
can improve from experience become increasingly valuable over time, developing expertise that static systems cannot match. This enables them to handle novel situations and continuously optimize their performance.

In [1]:
print("Tutorial 12: Learning from Experience - Adaptive Agents")
print("=" * 55)
print()
print("Building agents that learn, adapt, and improve over time...")
print()

Tutorial 12: Learning from Experience - Adaptive Agents

Building agents that learn, adapt, and improve over time...



In [3]:
# Essential imports
import uuid
import time
import math
import random
import pickle
import json
import threading
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Tuple, Callable, Union
from enum import Enum
from collections import defaultdict, deque
from abc import ABC, abstractmethod
import numpy as np
from pathlib import Path

In [4]:
# CORE LEARNING TYPES AND CONCEPTS
# =============================================================================

class LearningType(Enum):
    """Types of learning experiences"""
    SUPERVISED = "supervised"           # Learning from labeled examples
    UNSUPERVISED = "unsupervised"      # Finding patterns in unlabeled data
    REINFORCEMENT = "reinforcement"    # Learning from rewards/penalties
    IMITATION = "imitation"            # Learning by observing others
    TRANSFER = "transfer"              # Applying knowledge from other domains

class ExperienceType(Enum):
    """Categories of experiences for learning"""
    SUCCESS = "success"                # Successful task completion
    FAILURE = "failure"                # Failed attempts
    FEEDBACK = "feedback"              # External feedback/correction
    OBSERVATION = "observation"        # Observing other agents
    EXPLORATION = "exploration"        # Trying new approaches

class LearningObjective(Enum):
    """What the agent is trying to optimize"""
    ACCURACY = "accuracy"              # Correctness of responses
    EFFICIENCY = "efficiency"          # Speed of task completion
    QUALITY = "quality"                # Quality of outputs
    ADAPTABILITY = "adaptability"      # Ability to handle new situations
    ROBUSTNESS = "robustness"          # Consistency across conditions

@dataclass
class LearningExperience:
    """
    A single learning experience with context and outcomes

    This represents one instance of learning - what happened,
    what was learned, and how it can be applied in the future.
    """
    id: str
    experience_type: ExperienceType
    learning_type: LearningType

    # Context of the experience
    context: Dict[str, Any] = field(default_factory=dict)
    task_description: str = ""
    input_data: Any = None

    # Actions and outcomes
    action_taken: str = ""
    outcome: Any = None
    success: bool = False
    performance_metrics: Dict[str, float] = field(default_factory=dict)

    # Learning content
    lesson_learned: str = ""
    patterns_identified: List[str] = field(default_factory=list)
    knowledge_gained: Dict[str, Any] = field(default_factory=dict)

    # Metadata
    timestamp: float = field(default_factory=time.time)
    confidence: float = 0.5
    importance: float = 0.5
    source_agent: str = ""

    def get_age(self) -> float:
        """Get experience age in days"""
        return (time.time() - self.timestamp) / (24 * 3600)

    def calculate_relevance(self, current_context: Dict[str, Any]) -> float:
        """Calculate how relevant this experience is to current context"""
        if not current_context or not self.context:
            return 0.0

        # Simple context matching
        common_keys = set(current_context.keys()) & set(self.context.keys())
        if not common_keys:
            return 0.0

        relevance = 0.0
        for key in common_keys:
            if current_context[key] == self.context[key]:
                relevance += 1.0

        return relevance / len(self.context)

@dataclass
class LearningPattern:
    """
    A learned pattern that can be applied to new situations

    This represents extracted knowledge that generalizes
    beyond specific experiences.
    """
    id: str
    pattern_type: str
    description: str

    # Pattern definition
    conditions: Dict[str, Any] = field(default_factory=dict)
    actions: List[str] = field(default_factory=list)
    expected_outcomes: Dict[str, float] = field(default_factory=dict)

    # Pattern statistics
    times_observed: int = 0
    times_applied: int = 0
    success_rate: float = 0.0
    confidence: float = 0.0

    # Supporting evidence
    supporting_experiences: List[str] = field(default_factory=list)
    last_updated: float = field(default_factory=time.time)

    def update_from_experience(self, experience: LearningExperience, outcome_success: bool):
        """Update pattern based on new experience"""
        self.times_observed += 1
        if outcome_success:
            self.times_applied += 1

        # Update success rate
        if self.times_observed > 0:
            self.success_rate = self.times_applied / self.times_observed

        # Update confidence based on number of observations
        self.confidence = min(1.0, self.times_observed / 10.0)

        # Add to supporting evidence
        self.supporting_experiences.append(experience.id)
        if len(self.supporting_experiences) > 20:  # Keep only recent evidence
            self.supporting_experiences = self.supporting_experiences[-20:]

        self.last_updated = time.time()

# =============================================================================
# TEST THE FOUNDATION CLASSES
# =============================================================================

print("🧪 Testing foundation classes...")

# Create a sample learning experience
sample_experience = LearningExperience(
    id="exp_001",
    experience_type=ExperienceType.SUCCESS,
    learning_type=LearningType.SUPERVISED,
    context={'domain': 'customer_service', 'urgency': 'high'},
    task_description="Handle urgent customer complaint",
    action_taken="escalate_to_manager",
    success=True,
    performance_metrics={'satisfaction': 0.9, 'resolution_time': 0.8},
    lesson_learned="Escalation works well for urgent complaints",
    source_agent="service_bot"
)

print(f"✅ Created sample experience: {sample_experience.task_description}")
print(f"   Age: {sample_experience.get_age():.4f} days")
print(f"   Success: {sample_experience.success}")

# Create a sample learning pattern
sample_pattern = LearningPattern(
    id="pattern_001",
    pattern_type="success_pattern",
    description="Escalation pattern for urgent issues",
    conditions={'domain': 'customer_service', 'urgency': 'high'},
    actions=['escalate_to_manager'],
    times_observed=5,
    times_applied=4,
    confidence=0.8
)

print(f"✅ Created sample pattern: {sample_pattern.description}")
print(f"   Success rate: {sample_pattern.success_rate:.1%}")
print(f"   Confidence: {sample_pattern.confidence:.1f}")

# Test relevance calculation
test_context = {'domain': 'customer_service', 'urgency': 'high', 'customer_type': 'premium'}
relevance = sample_experience.calculate_relevance(test_context)
print(f"✅ Relevance to test context: {relevance:.2f}")

print()
print("🎉 Foundation classes working correctly!")
print("   Ready for Part 2: Experience Memory System")
print()
print("📋 Summary of what we built:")
print("   • LearningExperience - Individual learning moments")
print("   • LearningPattern - Extracted knowledge patterns")
print("   • Learning types (supervised, reinforcement, etc.)")
print("   • Experience types (success, failure, feedback, etc.)")
print("   • Learning objectives (accuracy, efficiency, etc.)")

🧪 Testing foundation classes...
✅ Created sample experience: Handle urgent customer complaint
   Age: 0.0000 days
   Success: True
✅ Created sample pattern: Escalation pattern for urgent issues
   Success rate: 0.0%
   Confidence: 0.8
✅ Relevance to test context: 1.00

🎉 Foundation classes working correctly!
   Ready for Part 2: Experience Memory System

📋 Summary of what we built:
   • LearningExperience - Individual learning moments
   • LearningPattern - Extracted knowledge patterns
   • Learning types (supervised, reinforcement, etc.)
   • Experience types (success, failure, feedback, etc.)
   • Learning objectives (accuracy, efficiency, etc.)


In [5]:
print("Tutorial 12 Part 2: Experience Memory System")
print("=" * 45)
print("Building sophisticated memory for learning experiences...")
print()

# Make sure you've run Part 1 first!
# This part builds on the classes from Part 1

class ExperienceMemory:
    """
    Specialized memory system for learning experiences

    This manages the storage, retrieval, and organization
    of learning experiences for effective knowledge extraction.
    """

    def __init__(self, capacity: int = 1000):
        self.capacity = capacity
        self.experiences: Dict[str, LearningExperience] = {}
        self.patterns: Dict[str, LearningPattern] = {}

        # Indices for efficient retrieval
        self.type_index: Dict[ExperienceType, List[str]] = defaultdict(list)
        self.context_index: Dict[str, List[str]] = defaultdict(list)
        self.success_index: Dict[bool, List[str]] = defaultdict(list)
        self.temporal_index: List[Tuple[float, str]] = []  # (timestamp, experience_id)

        print(f"🧠 Experience memory initialized (capacity: {capacity})")

    def add_experience(self, experience: LearningExperience) -> str:
        """Add a new learning experience"""

        # Add to main storage
        self.experiences[experience.id] = experience

        # Update indices
        self.type_index[experience.experience_type].append(experience.id)
        self.success_index[experience.success].append(experience.id)
        self.temporal_index.append((experience.timestamp, experience.id))

        # Context indexing
        for key, value in experience.context.items():
            context_key = f"{key}:{value}"
            self.context_index[context_key].append(experience.id)

        # Maintain capacity
        if len(self.experiences) > self.capacity:
            self._evict_old_experiences()

        print(f"   📝 Stored experience: {experience.task_description[:50]}...")
        return experience.id

    def _evict_old_experiences(self):
        """Remove oldest experiences when at capacity"""
        # Sort by timestamp and remove oldest
        self.temporal_index.sort()
        to_remove = self.temporal_index[:len(self.temporal_index) - self.capacity + 100]

        for _, exp_id in to_remove:
            if exp_id in self.experiences:
                self._remove_from_indices(exp_id)
                del self.experiences[exp_id]

        # Update temporal index
        self.temporal_index = self.temporal_index[len(to_remove):]

    def _remove_from_indices(self, experience_id: str):
        """Remove experience from all indices"""
        experience = self.experiences.get(experience_id)
        if not experience:
            return

        # Remove from type index
        if experience_id in self.type_index[experience.experience_type]:
            self.type_index[experience.experience_type].remove(experience_id)

        # Remove from success index
        if experience_id in self.success_index[experience.success]:
            self.success_index[experience.success].remove(experience_id)

        # Remove from context index
        for key, value in experience.context.items():
            context_key = f"{key}:{value}"
            if experience_id in self.context_index[context_key]:
                self.context_index[context_key].remove(experience_id)

    def find_similar_experiences(self, context: Dict[str, Any],
                                experience_type: ExperienceType = None,
                                limit: int = 10) -> List[LearningExperience]:
        """Find experiences similar to given context"""

        candidate_ids = set()

        # Filter by experience type if specified
        if experience_type:
            candidate_ids = set(self.type_index[experience_type])
        else:
            candidate_ids = set(self.experiences.keys())

        # Filter by context similarity
        context_candidates = set()
        for key, value in context.items():
            context_key = f"{key}:{value}"
            context_candidates.update(self.context_index[context_key])

        if context_candidates:
            candidate_ids &= context_candidates

        # Score and rank by relevance
        scored_experiences = []
        for exp_id in candidate_ids:
            if exp_id in self.experiences:
                experience = self.experiences[exp_id]
                relevance = experience.calculate_relevance(context)
                scored_experiences.append((relevance, experience))

        # Sort by relevance and return top results
        scored_experiences.sort(key=lambda x: x[0], reverse=True)
        return [exp for _, exp in scored_experiences[:limit]]

    def get_success_patterns(self, min_observations: int = 3) -> List[LearningPattern]:
        """Extract success patterns from successful experiences"""

        successful_experiences = [
            self.experiences[exp_id]
            for exp_id in self.success_index[True]
            if exp_id in self.experiences
        ]

        if len(successful_experiences) < min_observations:
            return []

        # Group by similar contexts
        context_groups = defaultdict(list)
        for exp in successful_experiences:
            # Create a simplified context signature
            context_sig = tuple(sorted(exp.context.items()))
            context_groups[context_sig].append(exp)

        patterns = []
        for context_sig, group_experiences in context_groups.items():
            if len(group_experiences) >= min_observations:
                # Create pattern from this group
                pattern_id = f"pattern_{len(self.patterns)}"

                # Extract common elements
                common_actions = []
                action_counts = defaultdict(int)

                for exp in group_experiences:
                    if exp.action_taken:
                        action_counts[exp.action_taken] += 1

                # Actions that appear in most experiences
                for action, count in action_counts.items():
                    if count >= len(group_experiences) * 0.6:  # 60% threshold
                        common_actions.append(action)

                if common_actions:
                    pattern = LearningPattern(
                        id=pattern_id,
                        pattern_type="success_pattern",
                        description=f"Successful pattern with {len(group_experiences)} observations",
                        conditions=dict(context_sig),
                        actions=common_actions,
                        times_observed=len(group_experiences),
                        times_applied=len(group_experiences),
                        success_rate=1.0,
                        confidence=min(1.0, len(group_experiences) / 10.0),
                        supporting_experiences=[exp.id for exp in group_experiences]
                    )
                    patterns.append(pattern)
                    self.patterns[pattern_id] = pattern

        return patterns

    def get_statistics(self) -> Dict[str, Any]:
        """Get memory statistics"""
        total_experiences = len(self.experiences)
        if total_experiences == 0:
            return {'total_experiences': 0}

        # Type distribution
        type_dist = {}
        for exp_type, exp_list in self.type_index.items():
            type_dist[exp_type.value] = len(exp_list)

        # Success rate
        successful = len(self.success_index[True])
        success_rate = successful / total_experiences if total_experiences > 0 else 0.0

        # Age distribution
        current_time = time.time()
        ages = [(current_time - exp.timestamp) / (24 * 3600) for exp in self.experiences.values()]
        avg_age = sum(ages) / len(ages) if ages else 0.0

        return {
            'total_experiences': total_experiences,
            'success_rate': success_rate,
            'type_distribution': type_dist,
            'patterns_discovered': len(self.patterns),
            'average_age_days': avg_age,
            'capacity_utilization': total_experiences / self.capacity
        }

# =============================================================================
# TEST THE EXPERIENCE MEMORY SYSTEM
# =============================================================================

print("🧪 Testing Experience Memory System...")

# Create memory system
memory = ExperienceMemory(capacity=50)

# Create diverse sample experiences
sample_experiences = [
    LearningExperience(
        id="exp_001",
        experience_type=ExperienceType.SUCCESS,
        learning_type=LearningType.SUPERVISED,
        context={'domain': 'customer_service', 'urgency': 'high'},
        task_description="Handle urgent customer complaint",
        action_taken="escalate_to_manager",
        success=True,
        performance_metrics={'satisfaction': 0.9, 'resolution_time': 0.8}
    ),
    LearningExperience(
        id="exp_002",
        experience_type=ExperienceType.FAILURE,
        learning_type=LearningType.REINFORCEMENT,
        context={'domain': 'customer_service', 'urgency': 'low'},
        task_description="Handle routine customer inquiry",
        action_taken="provide_standard_response",
        success=False,
        performance_metrics={'satisfaction': 0.3, 'resolution_time': 0.6}
    ),
    LearningExperience(
        id="exp_003",
        experience_type=ExperienceType.SUCCESS,
        learning_type=LearningType.SUPERVISED,
        context={'domain': 'customer_service', 'urgency': 'medium'},
        task_description="Process customer refund request",
        action_taken="verify_and_approve",
        success=True,
        performance_metrics={'satisfaction': 0.85, 'resolution_time': 0.9}
    ),
    LearningExperience(
        id="exp_004",
        experience_type=ExperienceType.SUCCESS,
        learning_type=LearningType.SUPERVISED,
        context={'domain': 'customer_service', 'urgency': 'high'},
        task_description="Handle urgent billing issue",
        action_taken="escalate_to_manager",
        success=True,
        performance_metrics={'satisfaction': 0.8, 'resolution_time': 0.7}
    ),
    LearningExperience(
        id="exp_005",
        experience_type=ExperienceType.OBSERVATION,
        learning_type=LearningType.IMITATION,
        context={'domain': 'technical_support', 'complexity': 'high'},
        task_description="Debug complex technical issue",
        action_taken="systematic_diagnosis",
        success=True,
        performance_metrics={'problem_solved': 1.0, 'time_taken': 0.6}
    )
]

# Add experiences to memory
print("\n📝 Adding experiences to memory:")
for exp in sample_experiences:
    memory.add_experience(exp)

# Test memory retrieval
print("\n🔍 Testing memory retrieval:")

# Find similar experiences
similar_context = {'domain': 'customer_service', 'urgency': 'high'}
similar_experiences = memory.find_similar_experiences(similar_context, limit=5)
print(f"   Found {len(similar_experiences)} experiences for context: {similar_context}")

for exp in similar_experiences:
    print(f"     - {exp.task_description} (action: {exp.action_taken})")

# Find successful experiences
successful_experiences = memory.find_similar_experiences(
    context={'domain': 'customer_service'},
    experience_type=ExperienceType.SUCCESS,
    limit=3
)
print(f"\n   Found {len(successful_experiences)} successful customer service experiences")

# Test pattern discovery
print("\n🔍 Testing pattern discovery:")
patterns = memory.get_success_patterns(min_observations=2)
print(f"   Discovered {len(patterns)} success patterns")

for pattern in patterns:
    print(f"     - {pattern.description}")
    print(f"       Conditions: {pattern.conditions}")
    print(f"       Actions: {pattern.actions}")
    print(f"       Confidence: {pattern.confidence:.2f}")

# Get memory statistics
print("\n📊 Memory Statistics:")
stats = memory.get_statistics()
for key, value in stats.items():
    if isinstance(value, dict):
        print(f"   {key}:")
        for subkey, subvalue in value.items():
            print(f"     {subkey}: {subvalue}")
    else:
        if isinstance(value, float):
            print(f"   {key}: {value:.3f}")
        else:
            print(f"   {key}: {value}")

print()
print("🎉 Experience Memory System working correctly!")
print("   Ready for Part 3: Learning Engine")
print()
print("📋 Summary of what we built:")
print("   • ExperienceMemory - Efficient storage and retrieval")
print("   • Smart indexing by type, context, and success")
print("   • Pattern discovery from successful experiences")
print("   • Memory capacity management with eviction")
print("   • Similarity search and relevance scoring")

Tutorial 12 Part 2: Experience Memory System
Building sophisticated memory for learning experiences...

🧪 Testing Experience Memory System...
🧠 Experience memory initialized (capacity: 50)

📝 Adding experiences to memory:
   📝 Stored experience: Handle urgent customer complaint...
   📝 Stored experience: Handle routine customer inquiry...
   📝 Stored experience: Process customer refund request...
   📝 Stored experience: Handle urgent billing issue...
   📝 Stored experience: Debug complex technical issue...

🔍 Testing memory retrieval:
   Found 4 experiences for context: {'domain': 'customer_service', 'urgency': 'high'}
     - Handle urgent customer complaint (action: escalate_to_manager)
     - Handle urgent billing issue (action: escalate_to_manager)
     - Handle routine customer inquiry (action: provide_standard_response)
     - Process customer refund request (action: verify_and_approve)

   Found 3 successful customer service experiences

🔍 Testing pattern discovery:
   Discovered

In [10]:
# Run this after Parts 1 and 2 to build the core learning processing system

print("Tutorial 12 Part 3: Learning Engine")
print("=" * 35)
print("Building the core learning processing system...")
print()

# Make sure you've run Parts 1 and 2 first!

class LearningEngine:
    """
    Core learning engine that processes experiences and extracts knowledge

    This is the heart of the learning system - it takes raw experiences
    and transforms them into actionable knowledge and patterns.
    """

    def __init__(self, learning_objectives: List[LearningObjective] = None):
        self.learning_objectives = learning_objectives or [LearningObjective.ACCURACY]
        self.experience_memory = ExperienceMemory()
        self.learning_algorithms = {}
        self.performance_baselines = {}

        # Learning statistics
        self.learning_sessions = 0
        self.patterns_discovered = 0
        self.knowledge_items = 0

        self._initialize_algorithms()
        print(f"🎓 Learning engine initialized with {len(self.learning_objectives)} objectives")

    def _initialize_algorithms(self):
        """Initialize learning algorithms for different learning types"""
        self.learning_algorithms = {
            LearningType.SUPERVISED: self._supervised_learning,
            LearningType.UNSUPERVISED: self._unsupervised_learning,
            LearningType.REINFORCEMENT: self._reinforcement_learning,
            LearningType.IMITATION: self._imitation_learning,
            LearningType.TRANSFER: self._transfer_learning
        }

    def process_experience(self, experience: LearningExperience) -> Dict[str, Any]:
        """Process a single learning experience"""

        # Store the experience
        exp_id = self.experience_memory.add_experience(experience)

        # Apply appropriate learning algorithm
        learning_result = None
        if experience.learning_type in self.learning_algorithms:
            algorithm = self.learning_algorithms[experience.learning_type]
            learning_result = algorithm(experience)

        # Update performance tracking
        self._update_performance_tracking(experience)

        # Trigger pattern discovery if enough new experiences
        if len(self.experience_memory.experiences) % 10 == 0:
            self._discover_patterns()

        return {
            'experience_id': exp_id,
            'learning_result': learning_result,
            'patterns_updated': self.patterns_discovered,
            'learning_type': experience.learning_type.value
        }

    def _supervised_learning(self, experience: LearningExperience) -> Dict[str, Any]:
        """Process supervised learning experience"""

        if not experience.outcome or experience.input_data is None:
            return {'error': 'Insufficient data for supervised learning'}

        # Extract features from input and label from outcome
        features = self._extract_features(experience.input_data, experience.context)
        label = self._extract_label(experience.outcome)

        # Simple pattern-based supervised learning
        pattern_key = f"supervised_{hash(str(features))}"

        if pattern_key not in self.experience_memory.patterns:
            # Create new pattern
            pattern = LearningPattern(
                id=pattern_key,
                pattern_type="supervised_mapping",
                description=f"Input-output mapping from supervised learning",
                conditions=features,
                expected_outcomes={'predicted_label': label},
                times_observed=1,
                confidence=0.1
            )
            self.experience_memory.patterns[pattern_key] = pattern
        else:
            # Update existing pattern
            pattern = self.experience_memory.patterns[pattern_key]
            pattern.update_from_experience(experience, experience.success)

        return {
            'pattern_id': pattern_key,
            'features_extracted': len(features),
            'confidence': pattern.confidence
        }

    def _unsupervised_learning(self, experience: LearningExperience) -> Dict[str, Any]:
        """Process unsupervised learning experience"""

        # Look for patterns in similar experiences
        similar_experiences = self.experience_memory.find_similar_experiences(
            experience.context,
            experience_type=experience.experience_type,
            limit=20
        )

        if len(similar_experiences) < 3:
            return {'message': 'Insufficient data for pattern discovery'}

        # Cluster similar experiences
        clusters = self._cluster_experiences(similar_experiences)

        # Create patterns from clusters
        patterns_created = 0
        for cluster_id, cluster_experiences in clusters.items():
            if len(cluster_experiences) >= 3:
                pattern_id = f"unsupervised_cluster_{cluster_id}"

                # Extract common characteristics
                common_context = self._extract_common_context(cluster_experiences)
                common_actions = self._extract_common_actions(cluster_experiences)

                if common_context and common_actions:
                    pattern = LearningPattern(
                        id=pattern_id,
                        pattern_type="unsupervised_cluster",
                        description=f"Discovered pattern from {len(cluster_experiences)} similar experiences",
                        conditions=common_context,
                        actions=common_actions,
                        times_observed=len(cluster_experiences)
                    )
                    self.experience_memory.patterns[pattern_id] = pattern
                    patterns_created += 1

        return {
            'clusters_found': len(clusters),
            'patterns_created': patterns_created,
            'total_experiences_analyzed': len(similar_experiences)
        }

    def _reinforcement_learning(self, experience: LearningExperience) -> Dict[str, Any]:
        """Process reinforcement learning experience"""

        # Extract reward signal
        reward = self._calculate_reward(experience)

        # Update action-value estimates
        state_action_key = f"rl_{hash(str(experience.context))}_{experience.action_taken}"

        # Simple Q-learning update
        alpha = 0.1  # Learning rate
        current_q = self._get_q_value(state_action_key)
        new_q = current_q + alpha * (reward - current_q)

        # Store updated Q-value as a pattern
        pattern = LearningPattern(
            id=state_action_key,
            pattern_type="q_value",
            description=f"Action value for state-action pair",
            conditions=experience.context,
            actions=[experience.action_taken],
            expected_outcomes={'q_value': new_q, 'reward': reward},
            times_observed=1 if state_action_key not in self.experience_memory.patterns else self.experience_memory.patterns[state_action_key].times_observed + 1
        )

        self.experience_memory.patterns[state_action_key] = pattern

        return {
            'reward': reward,
            'q_value_updated': new_q,
            'action': experience.action_taken,
            'learning_rate': alpha
        }

    def _imitation_learning(self, experience: LearningExperience) -> Dict[str, Any]:
        """Process imitation learning experience"""

        if experience.experience_type != ExperienceType.OBSERVATION:
            return {'error': 'Imitation learning requires observation experiences'}

        # Extract behavior patterns from observed actions
        observed_action = experience.action_taken
        context = experience.context

        # Create imitation pattern
        pattern_id = f"imitation_{hash(str(context))}"

        if pattern_id in self.experience_memory.patterns:
            pattern = self.experience_memory.patterns[pattern_id]
            pattern.update_from_experience(experience, experience.success)
        else:
            pattern = LearningPattern(
                id=pattern_id,
                pattern_type="imitation_behavior",
                description=f"Behavior pattern learned through imitation",
                conditions=context,
                actions=[observed_action],
                times_observed=1,
                confidence=0.3  # Lower initial confidence for imitation
            )
            self.experience_memory.patterns[pattern_id] = pattern

        return {
            'pattern_id': pattern_id,
            'observed_action': observed_action,
            'context_complexity': len(context),
            'pattern_confidence': pattern.confidence
        }

    def _transfer_learning(self, experience: LearningExperience) -> Dict[str, Any]:
        """Process transfer learning experience"""

        source_domain = experience.context.get('source_domain')
        target_domain = experience.context.get('target_domain')

        if not source_domain or not target_domain:
            return {'error': 'Transfer learning requires source and target domains'}

        # Find patterns from source domain
        source_patterns = [
            pattern for pattern in self.experience_memory.patterns.values()
            if source_domain in str(pattern.conditions)
        ]

        # Adapt patterns to target domain
        adapted_patterns = 0
        for source_pattern in source_patterns:
            if source_pattern.success_rate > 0.6:  # Only transfer successful patterns

                # Create adapted pattern for target domain
                adapted_id = f"transfer_{source_pattern.id}_{target_domain}"
                adapted_conditions = dict(source_pattern.conditions)
                adapted_conditions['domain'] = target_domain

                adapted_pattern = LearningPattern(
                    id=adapted_id,
                    pattern_type="transfer_adapted",
                    description=f"Pattern transferred from {source_domain} to {target_domain}",
                    conditions=adapted_conditions,
                    actions=source_pattern.actions.copy(),
                    expected_outcomes=source_pattern.expected_outcomes.copy(),
                    confidence=source_pattern.confidence * 0.7  # Reduced confidence for transfer
                )

                self.experience_memory.patterns[adapted_id] = adapted_pattern
                adapted_patterns += 1

        return {
            'source_patterns_found': len(source_patterns),
            'patterns_adapted': adapted_patterns,
            'source_domain': source_domain,
            'target_domain': target_domain
        }

    def _extract_features(self, input_data: Any, context: Dict[str, Any]) -> Dict[str, Any]:
        """Extract features from input data and context"""
        features = {}

        # Basic feature extraction
        if isinstance(input_data, str):
            features['input_type'] = 'string'
            features['input_length'] = len(input_data)
            features['input_words'] = len(input_data.split())
        elif isinstance(input_data, (int, float)):
            features['input_type'] = 'number'
            features['input_value'] = input_data
        elif isinstance(input_data, dict):
            features['input_type'] = 'dict'
            features['input_keys'] = len(input_data)

        # Add context features
        for key, value in context.items():
            features[f"context_{key}"] = value

        return features

    def _extract_label(self, outcome: Any) -> Any:
        """Extract label from outcome"""
        if isinstance(outcome, dict) and 'label' in outcome:
            return outcome['label']
        elif isinstance(outcome, bool):
            return 'success' if outcome else 'failure'
        else:
            return str(outcome)

    def _cluster_experiences(self, experiences: List[LearningExperience]) -> Dict[int, List[LearningExperience]]:
        """Simple clustering of experiences based on context similarity"""
        clusters = defaultdict(list)

        for experience in experiences:
            # Simple clustering based on context hash
            context_signature = hash(str(sorted(experience.context.items())))
            cluster_id = abs(context_signature) % 5  # Create up to 5 clusters
            clusters[cluster_id].append(experience)

        return dict(clusters)

    def _extract_common_context(self, experiences: List[LearningExperience]) -> Dict[str, Any]:
        """Extract common context elements from a group of experiences"""
        if not experiences:
            return {}

        # Find keys that appear in all experiences
        common_keys = set(experiences[0].context.keys())
        for exp in experiences[1:]:
            common_keys &= set(exp.context.keys())

        # Extract values that are the same across all experiences
        common_context = {}
        for key in common_keys:
            values = [exp.context[key] for exp in experiences]
            if len(set(values)) == 1:  # All have the same value
                common_context[key] = values[0]

        return common_context

    def _extract_common_actions(self, experiences: List[LearningExperience]) -> List[str]:
        """Extract common actions from a group of experiences"""
        action_counts = defaultdict(int)

        for exp in experiences:
            if exp.action_taken:
                action_counts[exp.action_taken] += 1

        # Return actions that appear in at least 50% of experiences
        threshold = len(experiences) * 0.5
        return [action for action, count in action_counts.items() if count >= threshold]

    def _calculate_reward(self, experience: LearningExperience) -> float:
        """Calculate reward signal from experience"""
        if experience.success:
            base_reward = 1.0
        else:
            base_reward = -1.0

        # Adjust based on performance metrics
        if experience.performance_metrics:
            performance_bonus = sum(experience.performance_metrics.values()) / len(experience.performance_metrics)
            return base_reward + (performance_bonus - 0.5)  # Normalize around 0.5

        return base_reward

    def _get_q_value(self, state_action_key: str) -> float:
        """Get current Q-value for state-action pair"""
        if state_action_key in self.experience_memory.patterns:
            pattern = self.experience_memory.patterns[state_action_key]
            return pattern.expected_outcomes.get('q_value', 0.0)
        return 0.0

    def _update_performance_tracking(self, experience: LearningExperience):
        """Update performance baselines and tracking"""
        for objective in self.learning_objectives:
            if objective.value in experience.performance_metrics:
                value = experience.performance_metrics[objective.value]

                if objective not in self.performance_baselines:
                    self.performance_baselines[objective] = {'values': [], 'baseline': 0.0}

                self.performance_baselines[objective]['values'].append(value)

                # Keep only recent values
                if len(self.performance_baselines[objective]['values']) > 100:
                    self.performance_baselines[objective]['values'] = self.performance_baselines[objective]['values'][-100:]

                # Update baseline
                values = self.performance_baselines[objective]['values']
                self.performance_baselines[objective]['baseline'] = sum(values) / len(values)

    def _discover_patterns(self):
        """Trigger pattern discovery from accumulated experiences"""
        new_patterns = self.experience_memory.get_success_patterns()
        self.patterns_discovered += len(new_patterns)

        if new_patterns:
            print(f"   🔍 Discovered {len(new_patterns)} new patterns")

    def get_learning_recommendations(self, context: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Get learning-based recommendations for given context"""

        recommendations = []

        # Find relevant patterns
        for pattern in self.experience_memory.patterns.values():
            relevance = self._calculate_pattern_relevance(pattern, context)

            if relevance > 0.5 and pattern.confidence > 0.3:
                recommendation = {
                    'pattern_id': pattern.id,
                    'suggested_actions': pattern.actions,
                    'confidence': pattern.confidence,
                    'relevance': relevance,
                    'success_rate': pattern.success_rate,
                    'description': pattern.description
                }
                recommendations.append(recommendation)

        # Sort by relevance and confidence
        recommendations.sort(key=lambda x: x['relevance'] * x['confidence'], reverse=True)

        return recommendations[:5]  # Top 5 recommendations

    def _calculate_pattern_relevance(self, pattern: LearningPattern, context: Dict[str, Any]) -> float:
        """Calculate how relevant a pattern is to the current context"""
        if not pattern.conditions or not context:
            return 0.0

        matching_conditions = 0
        total_conditions = len(pattern.conditions)

        for key, value in pattern.conditions.items():
            if key in context and context[key] == value:
                matching_conditions += 1

        return matching_conditions / total_conditions if total_conditions > 0 else 0.0

    def get_learning_statistics(self) -> Dict[str, Any]:
        """Get comprehensive learning statistics"""
        memory_stats = self.experience_memory.get_statistics()

        # Performance trends
        performance_trends = {}
        for objective, data in self.performance_baselines.items():
            if len(data['values']) > 1:
                recent_avg = sum(data['values'][-10:]) / min(10, len(data['values']))
                overall_avg = data['baseline']
                trend = 'improving' if recent_avg > overall_avg else 'declining'
                performance_trends[objective.value] = {
                    'trend': trend,
                    'recent_average': recent_avg,
                    'overall_average': overall_avg
                }

        return {
            'learning_sessions': self.learning_sessions,
            'patterns_discovered': self.patterns_discovered,
            'knowledge_items': self.knowledge_items,
            'memory_statistics': memory_stats,
            'performance_trends': performance_trends,
            'learning_objectives': [obj.value for obj in self.learning_objectives]
        }

# =============================================================================
# TEST THE LEARNING ENGINE
# =============================================================================

print("🧪 Testing Learning Engine...")

# Create learning engine
learning_engine = LearningEngine([
    LearningObjective.ACCURACY,
    LearningObjective.EFFICIENCY,
    LearningObjective.QUALITY
])

# Test different types of learning
print("\n🎓 Testing different learning algorithms:")

# Test 1: Supervised Learning
print("\n1. Supervised Learning:")
supervised_exp = LearningExperience(
    id="supervised_001",
    experience_type=ExperienceType.SUCCESS,
    learning_type=LearningType.SUPERVISED,
    context={'domain': 'classification', 'task_type': 'email_categorization'},
    task_description="Categorize email as spam or not spam",
    input_data="Free money click here now!!!",
    outcome={'label': 'spam'},
    action_taken="classify_as_spam",
    success=True,
    performance_metrics={'accuracy': 0.95, 'confidence': 0.9}
)

result = learning_engine.process_experience(supervised_exp)
print(f"   Result: {result['learning_result']}")

# Test 2: Reinforcement Learning
print("\n2. Reinforcement Learning:")
rl_exp = LearningExperience(
    id="rl_001",
    experience_type=ExperienceType.SUCCESS,
    learning_type=LearningType.REINFORCEMENT,
    context={'domain': 'customer_service', 'situation': 'angry_customer'},
    task_description="Handle angry customer complaint",
    action_taken="apologize_and_escalate",
    success=True,
    performance_metrics={'customer_satisfaction': 0.8, 'resolution_time': 0.7}
)

result = learning_engine.process_experience(rl_exp)
print(f"   Result: {result['learning_result']}")

# Test 3: Imitation Learning
print("\n3. Imitation Learning:")
imitation_exp = LearningExperience(
    id="imitation_001",
    experience_type=ExperienceType.OBSERVATION,
    learning_type=LearningType.IMITATION,
    context={'domain': 'negotiation', 'expert_agent': 'senior_negotiator'},
    task_description="Observed expert negotiation technique",
    action_taken="build_rapport_before_negotiating",
    success=True,
    performance_metrics={'effectiveness_observed': 0.9}
)

result = learning_engine.process_experience(imitation_exp)
print(f"   Result: {result['learning_result']}")

# Test 4: Transfer Learning
print("\n4. Transfer Learning:")
transfer_exp = LearningExperience(
    id="transfer_001",
    experience_type=ExperienceType.EXPLORATION,
    learning_type=LearningType.TRANSFER,
    context={'source_domain': 'customer_service', 'target_domain': 'technical_support'},
    task_description="Transfer customer service skills to technical support",
    success=True
)

result = learning_engine.process_experience(transfer_exp)
print(f"   Result: {result['learning_result']}")

# Test recommendations
print("\n🔍 Testing learning recommendations:")
test_context = {'domain': 'customer_service', 'situation': 'complaint'}
recommendations = learning_engine.get_learning_recommendations(test_context)

print(f"   Found {len(recommendations)} recommendations for context: {test_context}")
for i, rec in enumerate(recommendations):
    print(f"   {i+1}. {rec['description']} (confidence: {rec['confidence']:.2f})")

# Get learning statistics
print("\n📊 Learning Engine Statistics:")
stats = learning_engine.get_learning_statistics()
for key, value in stats.items():
    if isinstance(value, dict):
        print(f"   {key}:")
        for subkey, subvalue in value.items():
            print(f"     {subkey}: {subvalue}")
    else:
        print(f"   {key}: {value}")

print()
print("🎉 Learning Engine working correctly!")
print("   Ready for Part 4: Adaptive Agent")
print()
print("📋 Summary of what we built:")
print("   • LearningEngine - Core learning processing system")
print("   • 5 learning algorithms (supervised, reinforcement, etc.)")
print("   • Pattern discovery and knowledge extraction")
print("   • Learning recommendations based on context")
print("   • Performance tracking and trend analysis")
print("   • Transfer learning between domains")

Tutorial 12 Part 3: Learning Engine
Building the core learning processing system...

🧪 Testing Learning Engine...
🧠 Experience memory initialized (capacity: 1000)
🎓 Learning engine initialized with 3 objectives

🎓 Testing different learning algorithms:

1. Supervised Learning:
   📝 Stored experience: Categorize email as spam or not spam...
   Result: {'pattern_id': 'supervised_-8260293994354950701', 'features_extracted': 5, 'confidence': 0.1}

2. Reinforcement Learning:
   📝 Stored experience: Handle angry customer complaint...
   Result: {'reward': 1.25, 'q_value_updated': 0.125, 'action': 'apologize_and_escalate', 'learning_rate': 0.1}

3. Imitation Learning:
   📝 Stored experience: Observed expert negotiation technique...
   Result: {'pattern_id': 'imitation_-4875633224272169905', 'observed_action': 'build_rapport_before_negotiating', 'context_complexity': 2, 'pattern_confidence': 0.3}

4. Transfer Learning:
   📝 Stored experience: Transfer customer service skills to technical supp.

In [11]:
# Run this after Parts 1, 2, and 3 to build an agent that learns and adapts

print("Tutorial 12 Part 4: Adaptive Agent")
print("=" * 35)
print("Building an agent that learns and adapts from experience...")
print()

# Make sure you've run Parts 1, 2, and 3 first!
# This builds on: LearningExperience, LearningPattern, ExperienceMemory, LearningEngine

class AdaptiveAgent:
    """
    An agent that learns and adapts from experience

    This agent continuously improves its performance by learning
    from every interaction and applying that knowledge to future tasks.

    Key features:
    - Learns from every task execution
    - Adapts exploration and confidence based on performance
    - Builds expertise in different domains
    - Can observe and learn from other agents
    - Integrates external feedback for improvement
    """

    def __init__(self, agent_id: str = None, name: str = "",
                 learning_objectives: List[LearningObjective] = None):
        self.id = agent_id or str(uuid.uuid4())
        self.name = name or f"adaptive_agent_{self.id[:8]}"

        # Learning system - this is the core of the adaptive behavior
        self.learning_engine = LearningEngine(learning_objectives or [LearningObjective.ACCURACY])

        # Expertise tracking - how good is the agent at different things?
        self.expertise_areas = defaultdict(float)  # Domain -> expertise level (0-1)

        # Performance and adaptation tracking
        self.task_history = []
        self.adaptation_history = []
        self.performance_metrics = defaultdict(list)

        # Behavioral parameters that adapt over time
        self.confidence_threshold = 0.6    # How confident to be before using learned knowledge
        self.exploration_rate = 0.1        # How often to try new approaches vs use known good ones
        self.adaptation_frequency = 10     # How often to adapt behavior (every N tasks)

        print(f"🤖 Created adaptive agent: {self.name}")
        print(f"   Learning objectives: {[obj.value for obj in self.learning_engine.learning_objectives]}")
        print(f"   Initial exploration rate: {self.exploration_rate}")
        print(f"   Initial confidence threshold: {self.confidence_threshold}")

    def execute_task(self, task_description: str, task_data: Any,
                    context: Dict[str, Any] = None) -> Dict[str, Any]:
        """
        Execute a task and learn from the experience

        This is where the magic happens - the agent:
        1. Gets recommendations from its learning engine
        2. Chooses an action (exploration vs exploitation)
        3. Executes the task
        4. Evaluates performance
        5. Creates a learning experience
        6. Updates its expertise and potentially adapts behavior
        """

        task_id = str(uuid.uuid4())
        start_time = time.time()
        context = context or {}

        print(f"🎯 {self.name} executing: {task_description[:50]}...")

        # Step 1: Get learning-based recommendations
        recommendations = self.learning_engine.get_learning_recommendations(context)
        print(f"   💡 Found {len(recommendations)} recommendations from past experience")

        # Step 2: Choose action based on recommendations and exploration strategy
        chosen_action = self._choose_action(recommendations, context)
        print(f"   🎲 Chose action: {chosen_action}")

        # Step 3: Execute the task (simulated)
        result = self._execute_action(chosen_action, task_data, context)

        # Step 4: Measure performance
        execution_time = time.time() - start_time
        performance = self._evaluate_performance(result, execution_time)

        success_indicator = "✅" if performance['success'] else "❌"
        print(f"   {success_indicator} Result: {performance['success']} (score: {performance['overall_score']:.3f})")

        # Step 5: Create learning experience
        experience = LearningExperience(
            id=str(uuid.uuid4()),
            experience_type=ExperienceType.SUCCESS if performance['success'] else ExperienceType.FAILURE,
            learning_type=LearningType.REINFORCEMENT,  # Learning from rewards/penalties
            context=context,
            task_description=task_description,
            input_data=task_data,
            action_taken=chosen_action,
            outcome=result,
            success=performance['success'],
            performance_metrics=performance['metrics'],
            source_agent=self.name
        )

        # Step 6: Learn from this experience
        learning_result = self.learning_engine.process_experience(experience)

        # Step 7: Update expertise in relevant domains
        self._update_expertise(context, performance)

        # Step 8: Check if it's time to adapt behavior
        self._check_adaptation_trigger()

        # Step 9: Record this task in history
        task_record = {
            'task_id': task_id,
            'description': task_description,
            'action_taken': chosen_action,
            'performance': performance,
            'learning_applied': len(recommendations) > 0,
            'timestamp': time.time()
        }
        self.task_history.append(task_record)

        return {
            'task_id': task_id,
            'result': result,
            'performance': performance,
            'action_taken': chosen_action,
            'learning_result': learning_result,
            'recommendations_used': len(recommendations),
            'execution_time': execution_time
        }

    def _choose_action(self, recommendations: List[Dict[str, Any]],
                      context: Dict[str, Any]) -> str:
        """
        Choose action based on recommendations and exploration strategy

        This implements the exploration vs exploitation tradeoff:
        - Exploration: Try new things to discover better approaches
        - Exploitation: Use known good approaches from past experience
        """

        # Exploration: Sometimes try random things to discover new approaches
        if random.random() < self.exploration_rate:
            possible_actions = [
                'systematic_approach',
                'creative_solution',
                'methodical_process',
                'innovative_technique',
                'standard_procedure'
            ]
            chosen = random.choice(possible_actions)
            print(f"     🔍 Exploring with action: {chosen}")
            return chosen

        # Exploitation: Use learned knowledge if we have good recommendations
        if recommendations:
            best_recommendation = recommendations[0]  # Recommendations are sorted by relevance

            # Only use recommendation if we're confident enough
            if best_recommendation['confidence'] > self.confidence_threshold:
                action = (best_recommendation['suggested_actions'][0]
                         if best_recommendation['suggested_actions']
                         else 'default_action')
                print(f"     🧠 Using learned approach: {action} (confidence: {best_recommendation['confidence']:.2f})")
                return action
            else:
                print(f"     🤔 Recommendation confidence too low: {best_recommendation['confidence']:.2f} < {self.confidence_threshold:.2f}")

        # Domain expertise: Use expert knowledge if we have it
        domain = context.get('domain', 'general')
        if domain in self.expertise_areas and self.expertise_areas[domain] > 0.5:
            expert_action = f"expert_{domain}_approach"
            print(f"     🎓 Using domain expertise: {expert_action} (expertise: {self.expertise_areas[domain]:.2f})")
            return expert_action

        # Fallback: Use standard approach
        print(f"     📝 Using standard approach (no strong recommendations or expertise)")
        return 'standard_approach'

    def _execute_action(self, action: str, task_data: Any, context: Dict[str, Any]) -> Dict[str, Any]:
        """
        Execute the chosen action (simulated)

        This simulates actually performing the task. In a real system,
        this would call actual APIs, execute code, etc.
        """

        # Define base effectiveness for different action types
        action_effectiveness = {
            'systematic_approach': 0.75,
            'creative_solution': 0.85,  # High risk, high reward
            'methodical_process': 0.70,
            'innovative_technique': 0.80,
            'standard_procedure': 0.60,
            'standard_approach': 0.50,
            'default_action': 0.40
        }

        # Expert actions are more effective in their domain
        if action.startswith('expert_'):
            domain = action.split('_')[1]
            base_effectiveness = 0.60
            expertise_bonus = self.expertise_areas.get(domain, 0.0) * 0.35  # Up to 35% bonus
            effectiveness = base_effectiveness + expertise_bonus
        else:
            effectiveness = action_effectiveness.get(action, 0.50)

        # Add some realistic randomness (real world is unpredictable)
        randomness = random.uniform(-0.15, 0.15)
        actual_effectiveness = max(0.0, min(1.0, effectiveness + randomness))

        # Generate realistic result based on effectiveness
        if actual_effectiveness > 0.8:
            status = "excellent"
            description = f"Excellently completed using {action}"
        elif actual_effectiveness > 0.6:
            status = "good"
            description = f"Successfully completed using {action}"
        elif actual_effectiveness > 0.4:
            status = "partial"
            description = f"Partially completed using {action} - some issues"
        else:
            status = "failed"
            description = f"Failed to complete using {action}"

        return {
            'status': status,
            'description': description,
            'quality_score': actual_effectiveness,
            'action_used': action,
            'details': f"Executed {action} with {actual_effectiveness:.1%} effectiveness"
        }

    def _evaluate_performance(self, result: Dict[str, Any], execution_time: float) -> Dict[str, Any]:
        """
        Evaluate how well the agent performed on this task

        This creates standardized performance metrics that can be
        compared across different tasks and used for learning.
        """

        quality_score = result.get('quality_score', 0.0)

        # Calculate multiple performance dimensions
        accuracy = quality_score  # How correct/good was the result
        efficiency = max(0.0, 1.0 - min(execution_time / 5.0, 1.0))  # Faster is better, cap at 5 seconds
        success = quality_score > 0.5  # Binary success/failure

        # Composite metrics
        overall_score = (accuracy * 0.5 + efficiency * 0.3 + quality_score * 0.2)

        metrics = {
            'accuracy': accuracy,
            'efficiency': efficiency,
            'quality': quality_score,
            'execution_time': execution_time
        }

        return {
            'success': success,
            'metrics': metrics,
            'overall_score': overall_score
        }

    def _update_expertise(self, context: Dict[str, Any], performance: Dict[str, Any]):
        """
        Update expertise levels based on task performance

        This is how the agent builds up specialized knowledge in different areas.
        Better performance in a domain increases expertise in that domain.
        """

        domain = context.get('domain', 'general')
        task_type = context.get('task_type', 'general')
        performance_score = performance['overall_score']

        # Learning rate - how quickly expertise changes
        learning_rate = 0.1

        # Update domain expertise using exponential moving average
        current_domain_expertise = self.expertise_areas[domain]
        new_domain_expertise = current_domain_expertise + learning_rate * (performance_score - current_domain_expertise)
        self.expertise_areas[domain] = max(0.0, min(1.0, new_domain_expertise))

        # Also update task type expertise if it's different from domain
        if task_type != 'general' and task_type != domain:
            current_task_expertise = self.expertise_areas[task_type]
            new_task_expertise = current_task_expertise + learning_rate * (performance_score - current_task_expertise)
            self.expertise_areas[task_type] = max(0.0, min(1.0, new_task_expertise))

        # Track performance metrics for trend analysis
        for metric_name, value in performance['metrics'].items():
            self.performance_metrics[metric_name].append(value)

            # Keep only recent values to save memory
            if len(self.performance_metrics[metric_name]) > 100:
                self.performance_metrics[metric_name] = self.performance_metrics[metric_name][-100:]

        # Log significant expertise changes
        expertise_change = abs(new_domain_expertise - current_domain_expertise)
        if expertise_change > 0.05:  # Significant change
            print(f"     📈 {domain} expertise: {current_domain_expertise:.2f} → {new_domain_expertise:.2f}")

    def _check_adaptation_trigger(self):
        """
        Check if it's time to adapt behavior based on recent performance

        The agent periodically reviews its performance and adjusts its
        exploration rate and confidence threshold accordingly.
        """

        if len(self.task_history) % self.adaptation_frequency == 0 and len(self.task_history) > 0:
            print(f"     🔄 Triggering adaptation check (task #{len(self.task_history)})")
            self._adapt_behavior()

    def _adapt_behavior(self):
        """
        Adapt behavior based on recent performance

        This is the heart of the adaptive behavior - the agent looks at
        how it's been doing and adjusts its strategy accordingly.
        """

        if len(self.task_history) < 5:
            return  # Need some history to make adaptations

        # Analyze recent performance (last 10 tasks or all if less)
        recent_tasks = self.task_history[-10:]
        recent_success_rate = sum(1 for task in recent_tasks if task['performance']['success']) / len(recent_tasks)
        recent_avg_score = sum(task['performance']['overall_score'] for task in recent_tasks) / len(recent_tasks)

        print(f"       📊 Recent performance: {recent_success_rate:.1%} success, {recent_avg_score:.3f} avg score")

        # Store original values for comparison
        old_exploration = self.exploration_rate
        old_confidence = self.confidence_threshold

        # Adapt exploration rate based on success
        if recent_success_rate > 0.8:
            # High success rate - reduce exploration, exploit current knowledge
            self.exploration_rate = max(0.05, self.exploration_rate * 0.9)
            print(f"       ✅ High success rate - reducing exploration")
        elif recent_success_rate < 0.4:
            # Low success rate - increase exploration, try new things
            self.exploration_rate = min(0.3, self.exploration_rate * 1.1)
            print(f"       ❌ Low success rate - increasing exploration")

        # Adapt confidence threshold based on average performance
        if recent_avg_score > 0.7:
            # Good performance - be less conservative, trust recommendations more
            self.confidence_threshold = max(0.4, self.confidence_threshold - 0.05)
            print(f"       🎯 Good performance - lowering confidence threshold")
        elif recent_avg_score < 0.4:
            # Poor performance - be more conservative, require higher confidence
            self.confidence_threshold = min(0.8, self.confidence_threshold + 0.05)
            print(f"       🛡️ Poor performance - raising confidence threshold")

        # Record this adaptation
        adaptation_record = {
            'timestamp': time.time(),
            'trigger_task': len(self.task_history),
            'recent_success_rate': recent_success_rate,
            'recent_avg_score': recent_avg_score,
            'old_exploration_rate': old_exploration,
            'new_exploration_rate': self.exploration_rate,
            'old_confidence_threshold': old_confidence,
            'new_confidence_threshold': self.confidence_threshold
        }
        self.adaptation_history.append(adaptation_record)

        # Report changes
        exploration_change = self.exploration_rate - old_exploration
        confidence_change = self.confidence_threshold - old_confidence

        print(f"       🔧 Adaptation complete:")
        print(f"         Exploration: {old_exploration:.3f} → {self.exploration_rate:.3f} ({exploration_change:+.3f})")
        print(f"         Confidence: {old_confidence:.3f} → {self.confidence_threshold:.3f} ({confidence_change:+.3f})")

    def learn_from_feedback(self, task_id: str, feedback: Dict[str, Any]):
        """
        Learn from external feedback on a completed task

        This allows the agent to learn from human feedback or other
        external evaluations of its performance.
        """

        # Find the task that this feedback refers to
        task_record = None
        for task in self.task_history:
            if task['task_id'] == task_id:
                task_record = task
                break

        if not task_record:
            print(f"   ⚠️ Task {task_id[:8]}... not found for feedback")
            return None

        print(f"   📝 Processing feedback for task: {task_record['description'][:40]}...")

        # Create a supervised learning experience from the feedback
        experience = LearningExperience(
            id=str(uuid.uuid4()),
            experience_type=ExperienceType.FEEDBACK,
            learning_type=LearningType.SUPERVISED,  # Learning from labeled examples
            context={'task_type': 'feedback_learning', 'original_task': task_id},
            task_description=f"Feedback on: {task_record['description']}",
            input_data=task_record['action_taken'],
            outcome=feedback,
            success=feedback.get('satisfactory', False),
            performance_metrics=feedback.get('metrics', {}),
            lesson_learned=feedback.get('lesson', ''),
            source_agent='external_feedback'
        )

        # Process the feedback as a learning experience
        learning_result = self.learning_engine.process_experience(experience)

        feedback_quality = "positive" if feedback.get('satisfactory', False) else "corrective"
        print(f"   ✅ Learned from {feedback_quality} feedback")

        return learning_result

    def observe_other_agent(self, other_agent: 'AdaptiveAgent', task_description: str,
                           context: Dict[str, Any] = None):
        """
        Learn by observing another agent's behavior

        This implements imitation learning - the agent can watch
        what other successful agents do and learn from them.
        """

        context = context or {}
        print(f"   👁️ Observing {other_agent.name} on task: {task_description[:40]}...")

        # Get what the other agent would recommend for this context
        other_recommendations = other_agent.learning_engine.get_learning_recommendations(context)

        if other_recommendations:
            # Learn from the other agent's best approach
            best_approach = other_recommendations[0]

            # Create an imitation learning experience
            experience = LearningExperience(
                id=str(uuid.uuid4()),
                experience_type=ExperienceType.OBSERVATION,
                learning_type=LearningType.IMITATION,
                context=context,
                task_description=f"Observed {other_agent.name}: {task_description}",
                action_taken=best_approach['suggested_actions'][0] if best_approach['suggested_actions'] else 'unknown_action',
                success=True,  # Assume observed behavior is good to imitate
                confidence=best_approach['confidence'],
                source_agent=other_agent.name,
                lesson_learned=f"Learned approach from {other_agent.name}"
            )

            # Process the observation as a learning experience
            learning_result = self.learning_engine.process_experience(experience)

            print(f"   ✅ Learned approach: {experience.action_taken} (confidence: {best_approach['confidence']:.2f})")
            return learning_result
        else:
            print(f"   ℹ️ No clear approach to learn from {other_agent.name}")
            return None

    def transfer_knowledge_to_domain(self, source_domain: str, target_domain: str):
        """
        Transfer learned knowledge from one domain to another

        This allows the agent to apply knowledge learned in one area
        to a new, related area (like applying customer service skills
        to technical support).
        """

        if source_domain not in self.expertise_areas:
            print(f"   ⚠️ No expertise in source domain: {source_domain}")
            return None

        source_expertise = self.expertise_areas[source_domain]
        print(f"   🔄 Transferring knowledge: {source_domain} ({source_expertise:.2f}) → {target_domain}")

        # Create a transfer learning experience
        experience = LearningExperience(
            id=str(uuid.uuid4()),
            experience_type=ExperienceType.EXPLORATION,
            learning_type=LearningType.TRANSFER,
            context={
                'source_domain': source_domain,
                'target_domain': target_domain,
                'source_expertise': source_expertise
            },
            task_description=f"Transfer knowledge from {source_domain} to {target_domain}",
            success=True,
            source_agent=self.name
        )

        # Process the transfer learning
        learning_result = self.learning_engine.process_experience(experience)

        # Initialize target domain expertise based on source (partial transfer)
        transfer_factor = 0.3  # 30% of source expertise transfers initially
        initial_target_expertise = source_expertise * transfer_factor
        current_target = self.expertise_areas[target_domain]
        self.expertise_areas[target_domain] = max(current_target, initial_target_expertise)

        new_target_expertise = self.expertise_areas[target_domain]
        print(f"   ✅ Transfer complete: {target_domain} expertise now {new_target_expertise:.2f}")

        return learning_result

    def get_agent_status(self) -> Dict[str, Any]:
        """
        Get comprehensive status of the agent including learning progress

        This provides a complete picture of how the agent is doing,
        what it has learned, and how it has adapted.
        """

        learning_stats = self.learning_engine.get_learning_statistics()

        # Calculate performance trends from recent tasks
        recent_tasks = self.task_history[-20:] if len(self.task_history) >= 20 else self.task_history
        if recent_tasks:
            avg_performance = sum(task['performance']['overall_score'] for task in recent_tasks) / len(recent_tasks)
            success_rate = sum(1 for task in recent_tasks if task['performance']['success']) / len(recent_tasks)
        else:
            avg_performance = 0.0
            success_rate = 0.0

        # Get top expertise areas
        top_expertise = sorted(self.expertise_areas.items(), key=lambda x: x[1], reverse=True)[:5]

        return {
            'agent_id': self.id,
            'name': self.name,
            'tasks_completed': len(self.task_history),
            'current_performance': {
                'average_score': avg_performance,
                'success_rate': success_rate,
                'exploration_rate': self.exploration_rate,
                'confidence_threshold': self.confidence_threshold
            },
            'expertise_areas': dict(top_expertise),
            'learning_progress': learning_stats,
            'adaptations_made': len(self.adaptation_history),
            'behavioral_parameters': {
                'exploration_rate': self.exploration_rate,
                'confidence_threshold': self.confidence_threshold,
                'adaptation_frequency': self.adaptation_frequency
            }
        }

# =============================================================================
# TEST THE ADAPTIVE AGENT WITH COMPREHENSIVE EXAMPLES
# =============================================================================

print("🧪 Testing Adaptive Agent with Comprehensive Examples...")
print()

# Create an adaptive agent for testing
test_agent = AdaptiveAgent(
    name="TestBot",
    learning_objectives=[LearningObjective.ACCURACY, LearningObjective.EFFICIENCY, LearningObjective.QUALITY]
)

# Create diverse and realistic test scenarios
test_scenarios = [
    {
        'description': 'Handle urgent customer complaint about billing error',
        'data': {
            'customer_tier': 'premium',
            'issue_severity': 'high',
            'previous_contacts': 2,
            'amount_disputed': 150.00
        },
        'context': {
            'domain': 'customer_service',
            'task_type': 'complaint_resolution',
            'urgency': 'high'
        }
    },
    {
        'description': 'Optimize slow database query for user dashboard',
        'data': {
            'query_type': 'complex_join',
            'table_sizes': ['large', 'medium', 'small'],
            'current_execution_time': 8.5,
            'target_time': 2.0
        },
        'context': {
            'domain': 'database_optimization',
            'task_type': 'performance_tuning',
            'complexity': 'high'
        }
    },
    {
        'description': 'Design responsive layout for mobile e-commerce app',
        'data': {
            'target_devices': ['smartphone', 'tablet'],
            'key_features': ['product_grid', 'search', 'checkout'],
            'brand_guidelines': 'modern_minimalist'
        },
        'context': {
            'domain': 'ui_design',
            'task_type': 'responsive_design',
            'platform': 'mobile'
        }
    },
    {
        'description': 'Analyze quarterly sales data for trends and insights',
        'data': {
            'data_period': 'Q1_2024',
            'data_sources': ['crm', 'analytics', 'finance'],
            'metrics_requested': ['growth', 'customer_acquisition', 'churn']
        },
        'context': {
            'domain': 'data_analysis',
            'task_type': 'business_intelligence',
            'scope': 'quarterly_review'
        }
    },
    {
        'description': 'Write API documentation for new authentication service',
        'data': {
            'api_type': 'REST',
            'endpoints_count': 12,
            'authentication_methods': ['oauth2', 'api_key'],
            'target_audience': 'external_developers'
        },
        'context': {
            'domain': 'technical_writing',
            'task_type': 'api_documentation',
            'complexity': 'medium'
        }
    },
    {
        'description': 'Debug memory leak in production web application',
        'data': {
            'application_type': 'nodejs_web_app',
            'symptoms': ['increasing_memory_usage', 'slow_response_times'],
            'affected_users': 'approximately_500',
            'business_impact': 'high'
        },
        'context': {
            'domain': 'software_debugging',
            'task_type': 'production_issue',
            'urgency': 'critical'
        }
    }
]

print(f"🎯 Running {len(test_scenarios)} diverse scenarios to test learning and adaptation...")
print()

# Execute scenarios and track learning progression
results_summary = []

for i, scenario in enumerate(test_scenarios):
    print(f"📋 Scenario {i+1}: {scenario['description']}")
    print(f"   Domain: {scenario['context']['domain']}, Type: {scenario['context']['task_type']}")

    # Execute the task
    result = test_agent.execute_task(
        task_description=scenario['description'],
        task_data=scenario['data'],
        context=scenario['context']
    )

    # Store result for analysis
    results_summary.append({
        'scenario': i+1,
        'domain': scenario['context']['domain'],
        'success': result['performance']['success'],
        'score': result['performance']['overall_score'],
        'action': result['action_taken'],
        'recommendations_used': result['recommendations_used']
    })

    # Show key results
    print(f"   ➤ Action taken: {result['action_taken']}")
    print(f"   ➤ Success: {'✅' if result['performance']['success'] else '❌'}")
    print(f"   ➤ Performance score: {result['performance']['overall_score']:.3f}")
    print(f"   ➤ Used {result['recommendations_used']} recommendations from experience")
    print()

    # Simulate external feedback on some tasks
    if i in [1, 3, 5]:  # Provide feedback on scenarios 2, 4, and 6
        feedback_type = "positive" if result['performance']['success'] else "corrective"
        feedback = {
            'satisfactory': result['performance']['success'],
            'metrics': {
                'expert_rating': random.uniform(0.7, 0.95) if result['performance']['success'] else random.uniform(0.3, 0.6),
                'efficiency_rating': random.uniform(0.6, 0.9)
            },
            'lesson': f"Scenario {i+1}: {'Well executed approach' if result['performance']['success'] else 'Consider alternative strategies for better results'}"
        }

        test_agent.learn_from_feedback(result['task_id'], feedback)
        print(f"   📝 Received {feedback_type} feedback: {feedback['lesson']}")
        print()

# Test knowledge transfer between domains
print("🔄 Testing knowledge transfer between domains...")

# Check current expertise levels
print("   Current expertise levels:")
for domain, level in sorted(test_agent.expertise_areas.items(), key=lambda x: x[1], reverse=True):
    if level > 0.1:  # Only show domains with meaningful expertise
        print(f"     {domain}: {level:.3f}")

# Transfer knowledge from highest expertise domain to a new domain
if test_agent.expertise_areas:
    source_domain = max(test_agent.expertise_areas.items(), key=lambda x: x[1])[0]
    target_domain = "cross_functional_collaboration"

    print(f"\n   Transferring knowledge: {source_domain} → {target_domain}")
    test_agent.transfer_knowledge_to_domain(source_domain, target_domain)

# Test observation learning
print("\n👁️ Testing observation learning...")

# Create a more experienced "teacher" agent
teacher_agent = AdaptiveAgent(name="ExperiencedBot")
teacher_agent.expertise_areas['project_management'] = 0.8
teacher_agent.expertise_areas['team_leadership'] = 0.7

# Student agent observes teacher
print("   Student agent observing teacher on project management task...")
test_agent.observe_other_agent(
    teacher_agent,
    "Plan project timeline and coordinate team resources",
    {'domain': 'project_management', 'complexity': 'high', 'team_size': 8}
)

# Show final agent status and learning summary
print("\n📊 FINAL AGENT STATUS AND LEARNING ANALYSIS")
print("=" * 55)

status = test_agent.get_agent_status()

print(f"🤖 Agent: {status['name']}")
print(f"   📈 Tasks completed: {status['tasks_completed']}")
print(f"   ✅ Current success rate: {status['current_performance']['success_rate']:.1%}")
print(f"   🎯 Average performance score: {status['current_performance']['average_score']:.3f}")
print(f"   🔍 Current exploration rate: {status['current_performance']['exploration_rate']:.3f}")
print(f"   🛡️ Current confidence threshold: {status['current_performance']['confidence_threshold']:.3f}")
print(f"   🔄 Total adaptations made: {status['adaptations_made']}")

print(f"\n🏆 TOP EXPERTISE AREAS:")
expertise_items = list(status['expertise_areas'].items())
if expertise_items:
    for i, (area, level) in enumerate(expertise_items[:5], 1):
        expertise_bar = "█" * int(level * 20) + "░" * (20 - int(level * 20))
        print(f"   {i}. {area}: {level:.3f} {expertise_bar}")
else:
    print("   No significant expertise developed yet")

print(f"\n📚 LEARNING PROGRESS:")
learning_progress = status['learning_progress']
print(f"   🔍 Patterns discovered: {learning_progress['patterns_discovered']}")
print(f"   💾 Total experiences stored: {learning_progress['memory_statistics']['total_experiences']}")
print(f"   ✅ Experience success rate: {learning_progress['memory_statistics']['success_rate']:.1%}")
print(f"   📊 Memory utilization: {learning_progress['memory_statistics']['capacity_utilization']:.1%}")

# Analyze learning progression through the scenarios
print(f"\n📈 LEARNING PROGRESSION ANALYSIS:")
print(f"   Scenario progression:")

for i, result in enumerate(results_summary):
    trend_indicator = ""
    if i > 0:
        prev_score = results_summary[i-1]['score']
        if result['score'] > prev_score + 0.1:
            trend_indicator = "↗️"
        elif result['score'] < prev_score - 0.1:
            trend_indicator = "↘️"
        else:
            trend_indicator = "➡️"

    success_icon = "✅" if result['success'] else "❌"
    print(f"     {result['scenario']}. {result['domain']}: {result['score']:.3f} {success_icon} {trend_indicator}")

# Show adaptation history
if test_agent.adaptation_history:
    print(f"\n🔄 BEHAVIORAL ADAPTATION HISTORY:")
    print(f"   The agent adapted its behavior {len(test_agent.adaptation_history)} times:")

    for i, adaptation in enumerate(test_agent.adaptation_history, 1):
        exploration_change = adaptation['new_exploration_rate'] - adaptation['old_exploration_rate']
        confidence_change = adaptation['new_confidence_threshold'] - adaptation['old_confidence_threshold']

        print(f"     {i}. After task #{adaptation['trigger_task']}:")
        print(f"        Success rate was {adaptation['recent_success_rate']:.1%}")
        print(f"        Exploration: {adaptation['old_exploration_rate']:.3f} → {adaptation['new_exploration_rate']:.3f} ({exploration_change:+.3f})")
        print(f"        Confidence: {adaptation['old_confidence_threshold']:.3f} → {adaptation['new_confidence_threshold']:.3f} ({confidence_change:+.3f})")

# Performance trend analysis
print(f"\n📊 PERFORMANCE TREND ANALYSIS:")
if len(results_summary) >= 3:
    early_performance = sum(r['score'] for r in results_summary[:3]) / 3
    late_performance = sum(r['score'] for r in results_summary[-3:]) / 3
    improvement = late_performance - early_performance

    trend_description = "improving" if improvement > 0.1 else "declining" if improvement < -0.1 else "stable"
    print(f"   Early performance (first 3 tasks): {early_performance:.3f}")
    print(f"   Recent performance (last 3 tasks): {late_performance:.3f}")
    print(f"   Overall trend: {trend_description} ({improvement:+.3f})")

    # Success rate trends
    early_success = sum(1 for r in results_summary[:3] if r['success']) / 3
    late_success = sum(1 for r in results_summary[-3:] if r['success']) / 3
    success_improvement = late_success - early_success
    print(f"   Success rate improvement: {early_success:.1%} → {late_success:.1%} ({success_improvement:+.1%})")

# Domain expertise analysis
print(f"\n🎯 DOMAIN EXPERTISE ANALYSIS:")
domain_tasks = defaultdict(list)
for result in results_summary:
    domain_tasks[result['domain']].append(result)

for domain, tasks in domain_tasks.items():
    if len(tasks) > 1:
        scores = [t['score'] for t in tasks]
        avg_score = sum(scores) / len(scores)
        success_rate = sum(1 for t in tasks if t['success']) / len(tasks)
        expertise_level = test_agent.expertise_areas.get(domain, 0.0)

        print(f"   {domain}:")
        print(f"     Tasks: {len(tasks)}, Avg score: {avg_score:.3f}, Success: {success_rate:.1%}")
        print(f"     Expertise developed: {expertise_level:.3f}")

# Recommendations usage analysis
total_recommendations = sum(r['recommendations_used'] for r in results_summary)
tasks_with_recommendations = sum(1 for r in results_summary if r['recommendations_used'] > 0)

print(f"\n🧠 LEARNING APPLICATION ANALYSIS:")
print(f"   Total recommendations used: {total_recommendations}")
print(f"   Tasks that used recommendations: {tasks_with_recommendations}/{len(results_summary)}")
print(f"   Average recommendations per task: {total_recommendations/len(results_summary):.1f}")

if tasks_with_recommendations > 0:
    # Analyze performance when using vs not using recommendations
    with_rec_scores = [r['score'] for r in results_summary if r['recommendations_used'] > 0]
    without_rec_scores = [r['score'] for r in results_summary if r['recommendations_used'] == 0]

    if with_rec_scores and without_rec_scores:
        with_rec_avg = sum(with_rec_scores) / len(with_rec_scores)
        without_rec_avg = sum(without_rec_scores) / len(without_rec_scores)

        print(f"   Performance with recommendations: {with_rec_avg:.3f}")
        print(f"   Performance without recommendations: {without_rec_avg:.3f}")
        print(f"   Learning benefit: {with_rec_avg - without_rec_avg:+.3f}")

print()
print("🎉 Adaptive Agent Testing Complete!")
print()
print("📋 WHAT WE DEMONSTRATED:")
print("   ✅ Task execution with learning-based action selection")
print("   ✅ Dynamic exploration vs exploitation balancing")
print("   ✅ Domain expertise development over time")
print("   ✅ Behavioral adaptation based on performance feedback")
print("   ✅ External feedback integration for supervised learning")
print("   ✅ Knowledge transfer between domains")
print("   ✅ Observation-based learning from other agents")
print("   ✅ Comprehensive performance tracking and analysis")
print()
print("🔑 KEY INSIGHTS FROM THIS TEST:")
print("   • The agent learned to make better decisions over time")
print("   • Exploration rate adapted based on success patterns")
print("   • Domain expertise accumulated through repeated tasks")
print("   • Behavioral parameters self-tuned for better performance")
print("   • Learning from multiple sources (experience, feedback, observation)")
print()
print("🚀 Ready for Part 5: Multi-Agent Learning System")
print("   Next we'll connect multiple adaptive agents for collaborative learning!")
print()
print("📦 WHAT WE BUILT IN PART 4:")
print("   • AdaptiveAgent class with comprehensive learning capabilities")
print("   • Dynamic action selection (exploration vs exploitation)")
print("   • Multi-dimensional performance evaluation")
print("   • Domain expertise tracking and development")
print("   • Behavioral adaptation based on performance trends")
print("   • External feedback integration")
print("   • Knowledge transfer between domains")
print("   • Observation-based learning from other agents")
print("   • Comprehensive status reporting and analysis")
print("   • Realistic task simulation and performance measurement")

Tutorial 12 Part 4: Adaptive Agent
Building an agent that learns and adapts from experience...

🧪 Testing Adaptive Agent with Comprehensive Examples...

🧠 Experience memory initialized (capacity: 1000)
🎓 Learning engine initialized with 3 objectives
🤖 Created adaptive agent: TestBot
   Learning objectives: ['accuracy', 'efficiency', 'quality']
   Initial exploration rate: 0.1
   Initial confidence threshold: 0.6
🎯 Running 6 diverse scenarios to test learning and adaptation...

📋 Scenario 1: Handle urgent customer complaint about billing error
   Domain: customer_service, Type: complaint_resolution
🎯 TestBot executing: Handle urgent customer complaint about billing err...
   💡 Found 0 recommendations from past experience
     📝 Using standard approach (no strong recommendations or expertise)
   🎲 Chose action: standard_approach
   ❌ Result: False (score: 0.647)
   📝 Stored experience: Handle urgent customer complaint about billing err...
     📈 customer_service expertise: 0.00 → 0.06
  

In [12]:
# Run this after Parts 1-4 to build coordinated learning across multiple agents

print("Tutorial 12 Part 5: Multi-Agent Learning System")
print("=" * 45)
print("Building coordinated learning across multiple agents...")
print()

# Make sure you've run Parts 1-4 first!

class MultiAgentLearningSystem:
    """
    System for coordinating learning across multiple adaptive agents

    This enables agents to learn from each other and share knowledge
    for collective improvement.
    """

    def __init__(self, name: str):
        self.name = name
        self.agents = {}
        self.shared_knowledge_base = {}
        self.collaboration_history = []
        self.knowledge_transfer_events = []

        print(f"🏫 Multi-agent learning system initialized: {name}")

    def add_agent(self, agent: AdaptiveAgent, specialization: str = None):
        """Add an adaptive agent to the learning system"""
        self.agents[agent.id] = {
            'agent': agent,
            'specialization': specialization,
            'joined_at': time.time(),
            'contributions': 0
        }
        print(f"   👥 Added agent {agent.name} (specialization: {specialization or 'general'})")

    def coordinate_learning_session(self, task_scenarios: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Coordinate a learning session across all agents"""

        print(f"🎓 Starting coordinated learning session with {len(task_scenarios)} scenarios...")

        session_results = {
            'session_id': str(uuid.uuid4()),
            'timestamp': time.time(),
            'scenarios_completed': 0,
            'agent_results': {},
            'knowledge_shared': 0,
            'performance_improvements': {}
        }

        # Execute scenarios
        for i, scenario in enumerate(task_scenarios):
            print(f"   📋 Scenario {i+1}: {scenario['description']}")

            scenario_results = {}

            # Each agent attempts the scenario
            for agent_id, agent_info in self.agents.items():
                agent = agent_info['agent']

                result = agent.execute_task(
                    task_description=scenario['description'],
                    task_data=scenario.get('data'),
                    context=scenario.get('context', {})
                )

                scenario_results[agent_id] = result

                if agent_id not in session_results['agent_results']:
                    session_results['agent_results'][agent_id] = []
                session_results['agent_results'][agent_id].append(result)

            # Cross-agent learning from this scenario
            self._facilitate_cross_learning(scenario_results, scenario)
            session_results['scenarios_completed'] += 1

        # Post-session knowledge consolidation
        knowledge_shared = self._consolidate_session_knowledge(session_results)
        session_results['knowledge_shared'] = knowledge_shared

        # Calculate performance improvements
        session_results['performance_improvements'] = self._calculate_improvements(session_results)

        print(f"   ✅ Learning session complete: {session_results['scenarios_completed']} scenarios, {knowledge_shared} knowledge transfers")

        return session_results

    def _facilitate_cross_learning(self, scenario_results: Dict[str, Any], scenario: Dict[str, Any]):
        """Facilitate learning between agents based on scenario results"""

        # Find best performing agent for this scenario
        best_agent_id = max(scenario_results.keys(),
                           key=lambda aid: scenario_results[aid]['performance']['overall_score'])
        best_result = scenario_results[best_agent_id]

        # Have other agents observe the best performer
        for agent_id, agent_info in self.agents.items():
            if agent_id != best_agent_id:
                observer_agent = agent_info['agent']
                best_agent = self.agents[best_agent_id]['agent']

                observer_agent.observe_other_agent(
                    best_agent,
                    scenario['description'],
                    scenario.get('context', {})
                )

        # Record collaboration
        collaboration_record = {
            'timestamp': time.time(),
            'scenario': scenario['description'],
            'best_performer': best_agent_id,
            'best_score': best_result['performance']['overall_score'],
            'observers': [aid for aid in self.agents.keys() if aid != best_agent_id]
        }
        self.collaboration_history.append(collaboration_record)

    def _consolidate_session_knowledge(self, session_results: Dict[str, Any]) -> int:
        """Consolidate knowledge from the learning session"""

        knowledge_transfers = 0

        # Extract successful patterns from all agents
        successful_patterns = {}

        for agent_id, results in session_results['agent_results'].items():
            agent = self.agents[agent_id]['agent']

            # Get patterns from successful tasks
            for result in results:
                if result['performance']['success']:
                    pattern_key = f"{result['action_taken']}_{agent_id}"
                    successful_patterns[pattern_key] = {
                        'agent_id': agent_id,
                        'action': result['action_taken'],
                        'performance': result['performance']['overall_score'],
                        'context': result.get('context', {})
                    }

        # Share successful patterns with underperforming agents
        for agent_id, agent_info in self.agents.items():
            agent = agent_info['agent']
            agent_results = session_results['agent_results'].get(agent_id, [])

            if agent_results:
                avg_performance = sum(r['performance']['overall_score'] for r in agent_results) / len(agent_results)

                # If performance is below average, share successful patterns
                if avg_performance < 0.6:
                    for pattern in successful_patterns.values():
                        if pattern['agent_id'] != agent_id and pattern['performance'] > avg_performance + 0.2:
                            # Create shared knowledge
                            knowledge_id = str(uuid.uuid4())
                            self.shared_knowledge_base[knowledge_id] = {
                                'pattern': pattern,
                                'created_at': time.time(),
                                'source_agent': pattern['agent_id'],
                                'shared_with': [agent_id],
                                'effectiveness': pattern['performance']
                            }
                            knowledge_transfers += 1

        return knowledge_transfers

    def _calculate_improvements(self, session_results: Dict[str, Any]) -> Dict[str, float]:
        """Calculate performance improvements for each agent"""

        improvements = {}

        for agent_id, results in session_results['agent_results'].items():
            if len(results) > 1:
                # Compare first half vs second half performance
                mid_point = len(results) // 2
                first_half_avg = sum(r['performance']['overall_score'] for r in results[:mid_point]) / mid_point
                second_half_avg = sum(r['performance']['overall_score'] for r in results[mid_point:]) / (len(results) - mid_point)

                improvement = second_half_avg - first_half_avg
                improvements[agent_id] = improvement

        return improvements

    def facilitate_knowledge_transfer(self, source_agent_id: str, target_agent_id: str,
                                    domain: str) -> Dict[str, Any]:
        """Facilitate direct knowledge transfer between two agents"""

        if source_agent_id not in self.agents or target_agent_id not in self.agents:
            return {'error': 'Agent not found'}

        source_agent = self.agents[source_agent_id]['agent']
        target_agent = self.agents[target_agent_id]['agent']

        # Check source agent's expertise in domain
        source_expertise = source_agent.expertise_areas.get(domain, 0.0)
        if source_expertise < 0.3:
            return {'error': f'Source agent has insufficient expertise in {domain}'}

        # Transfer knowledge
        transfer_result = target_agent.transfer_knowledge_to_domain(domain, domain)

        # Record transfer event
        transfer_event = {
            'timestamp': time.time(),
            'source_agent': source_agent_id,
            'target_agent': target_agent_id,
            'domain': domain,
            'source_expertise': source_expertise,
            'transfer_result': transfer_result
        }
        self.knowledge_transfer_events.append(transfer_event)

        # Update contribution counts
        self.agents[source_agent_id]['contributions'] += 1

        print(f"   🔄 Knowledge transfer: {source_agent.name} → {target_agent.name} (domain: {domain})")

        return {
            'success': True,
            'transfer_event': transfer_event,
            'source_expertise': source_expertise,
            'target_expertise_after': target_agent.expertise_areas.get(domain, 0.0)
        }

    def get_system_analytics(self) -> Dict[str, Any]:
        """Get comprehensive analytics for the learning system"""

        total_agents = len(self.agents)
        total_tasks = sum(len(agent_info['agent'].task_history) for agent_info in self.agents.values())
        total_collaborations = len(self.collaboration_history)
        total_transfers = len(self.knowledge_transfer_events)

        # Agent performance summary
        agent_performance = {}
        for agent_id, agent_info in self.agents.items():
            agent = agent_info['agent']
            status = agent.get_agent_status()
            agent_performance[agent_id] = {
                'name': agent.name,
                'tasks_completed': status['tasks_completed'],
                'success_rate': status['current_performance']['success_rate'],
                'top_expertise': list(status['expertise_areas'].keys())[:3],
                'contributions': agent_info['contributions']
            }

        # Knowledge sharing effectiveness
        if self.knowledge_transfer_events:
            recent_transfers = self.knowledge_transfer_events[-10:]
            avg_source_expertise = sum(t['source_expertise'] for t in recent_transfers) / len(recent_transfers)
        else:
            avg_source_expertise = 0.0

        return {
            'system_name': self.name,
            'total_agents': total_agents,
            'total_tasks_completed': total_tasks,
            'collaboration_events': total_collaborations,
            'knowledge_transfers': total_transfers,
            'shared_knowledge_items': len(self.shared_knowledge_base),
            'agent_performance': agent_performance,
            'knowledge_sharing_effectiveness': avg_source_expertise,
            'avg_tasks_per_agent': total_tasks / max(total_agents, 1)
        }

# =============================================================================
# TEST THE MULTI-AGENT LEARNING SYSTEM
# =============================================================================

print("🧪 Testing Multi-Agent Learning System...")

# Create multi-agent learning system
learning_system = MultiAgentLearningSystem("TechTeam Learning")

# Create specialized agents
agents = [
    AdaptiveAgent(name="BackendExpert", learning_objectives=[LearningObjective.EFFICIENCY, LearningObjective.ROBUSTNESS]),
    AdaptiveAgent(name="FrontendExpert", learning_objectives=[LearningObjective.QUALITY, LearningObjective.ADAPTABILITY]),
    AdaptiveAgent(name="DataExpert", learning_objectives=[LearningObjective.ACCURACY, LearningObjective.EFFICIENCY]),
    AdaptiveAgent(name="Generalist", learning_objectives=[LearningObjective.ADAPTABILITY])
]

# Add agents with specializations
specializations = ['backend', 'frontend', 'data_science', 'general']
for agent, specialization in zip(agents, specializations):
    learning_system.add_agent(agent, specialization)

# Pre-seed some expertise to make the demo more interesting
agents[0].expertise_areas['software_engineering'] = 0.7
agents[0].expertise_areas['database'] = 0.6
agents[1].expertise_areas['design'] = 0.8
agents[1].expertise_areas['technical_writing'] = 0.5
agents[2].expertise_areas['finance'] = 0.6
agents[2].expertise_areas['database'] = 0.7

print(f"\n🎯 Pre-seeded expertise:")
for i, agent in enumerate(agents):
    if agent.expertise_areas:
        top_expertise = sorted(agent.expertise_areas.items(), key=lambda x: x[1], reverse=True)[:2]
        print(f"   {agent.name}: {', '.join([f'{area}({level:.1f})' for area, level in top_expertise])}")

# Create diverse learning scenarios
learning_scenarios = [
    {
        'description': 'Process customer support request',
        'data': {'customer_type': 'premium', 'issue_type': 'technical', 'urgency': 'high'},
        'context': {'domain': 'customer_service', 'task_type': 'support'}
    },
    {
        'description': 'Optimize database query performance',
        'data': {'database_type': 'postgresql', 'query_complexity': 'high', 'data_size': 'large'},
        'context': {'domain': 'database', 'task_type': 'optimization'}
    },
    {
        'description': 'Design user interface for mobile application',
        'data': {'app_type': 'productivity', 'target_users': 'professionals', 'platform': 'ios'},
        'context': {'domain': 'design', 'task_type': 'ui_design'}
    },
    {
        'description': 'Analyze market data for investment decision',
        'data': {'market_trend': 'bullish', 'volatility': 'medium', 'sector': 'technology'},
        'context': {'domain': 'finance', 'task_type': 'analysis'}
    }
]

# Coordinate learning session
print(f"\n🎓 Running coordinated learning session...")
session_result = learning_system.coordinate_learning_session(learning_scenarios)

print(f"\n📈 SESSION RESULTS:")
print(f"   Scenarios Completed: {session_result['scenarios_completed']}")
print(f"   Knowledge Transfers: {session_result['knowledge_shared']}")
print(f"   Performance Improvements:")
for agent_id, improvement in session_result['performance_improvements'].items():
    agent_name = learning_system.agents[agent_id]['agent'].name
    trend = "↗️" if improvement > 0.1 else "↘️" if improvement < -0.1 else "➡️"
    print(f"     {agent_name}: {improvement:+.3f} {trend}")

# Test direct knowledge transfer
print(f"\n🔄 Testing direct knowledge transfer...")
backend_expert = agents[0]
data_expert = agents[2]

transfer_result = learning_system.facilitate_knowledge_transfer(
    backend_expert.id, data_expert.id, 'database'
)

if transfer_result.get('success'):
    print(f"   ✅ Transferred database knowledge from {backend_expert.name} to {data_expert.name}")
    print(f"   Source expertise: {transfer_result['source_expertise']:.2f}")
    print(f"   Target expertise after: {transfer_result['target_expertise_after']:.2f}")

# Get system analytics
print(f"\n📊 SYSTEM ANALYTICS:")
analytics = learning_system.get_system_analytics()
print(f"   Total Agents: {analytics['total_agents']}")
print(f"   Total Tasks: {analytics['total_tasks_completed']}")
print(f"   Collaboration Events: {analytics['collaboration_events']}")
print(f"   Knowledge Transfers: {analytics['knowledge_transfers']}")
print(f"   Shared Knowledge Items: {analytics['shared_knowledge_items']}")
print(f"   Avg Tasks per Agent: {analytics['avg_tasks_per_agent']:.1f}")

print(f"\n👥 AGENT PERFORMANCE SUMMARY:")
for agent_id, performance in analytics['agent_performance'].items():
    print(f"   {performance['name']}:")
    print(f"     Tasks: {performance['tasks_completed']}, Success: {performance['success_rate']:.1%}")
    print(f"     Top expertise: {', '.join(performance['top_expertise'])}")
    print(f"     Contributions: {performance['contributions']}")

# Show collaboration examples
if learning_system.collaboration_history:
    print(f"\n🤝 RECENT COLLABORATIONS:")
    for collab in learning_system.collaboration_history[-3:]:
        best_performer = learning_system.agents[collab['best_performer']]['agent'].name
        print(f"   {collab['scenario'][:40]}...")
        print(f"     Best performer: {best_performer} (score: {collab['best_score']:.3f})")
        print(f"     Observers: {len(collab['observers'])} agents")

print()
print("🎉 Multi-Agent Learning System working correctly!")
print("   Ready for Part 6: Learning Visualization and Analysis")
print()
print("📋 Summary of what we built:")
print("   • MultiAgentLearningSystem - Coordinated learning across agents")
print("   • Cross-agent observation and learning")
print("   • Knowledge consolidation and sharing")
print("   • Performance improvement tracking")
print("   • Direct knowledge transfer between agents")
print("   • Comprehensive system analytics")
print("   • Collaboration history and contribution tracking")

Tutorial 12 Part 5: Multi-Agent Learning System
Building coordinated learning across multiple agents...

🧪 Testing Multi-Agent Learning System...
🏫 Multi-agent learning system initialized: TechTeam Learning
🧠 Experience memory initialized (capacity: 1000)
🎓 Learning engine initialized with 2 objectives
🤖 Created adaptive agent: BackendExpert
   Learning objectives: ['efficiency', 'robustness']
   Initial exploration rate: 0.1
   Initial confidence threshold: 0.6
🧠 Experience memory initialized (capacity: 1000)
🎓 Learning engine initialized with 2 objectives
🤖 Created adaptive agent: FrontendExpert
   Learning objectives: ['quality', 'adaptability']
   Initial exploration rate: 0.1
   Initial confidence threshold: 0.6
🧠 Experience memory initialized (capacity: 1000)
🎓 Learning engine initialized with 2 objectives
🤖 Created adaptive agent: DataExpert
   Learning objectives: ['accuracy', 'efficiency']
   Initial exploration rate: 0.1
   Initial confidence threshold: 0.6
🧠 Experience memor

In [15]:
# Run this after Parts 1-5 to build comprehensive analysis and visualization tools

print("Tutorial 12 Part 6: Learning Visualization and Analysis")
print("=" * 55)
print("Building comprehensive analysis and visualization tools...")
print()

# Make sure you've run Parts 1-5 first!
# This builds on: AdaptiveAgent, MultiAgentLearningSystem, and all previous components

class LearningVisualizer:
    """
    Visualization and analysis tools for learning systems

    This provides insights into learning progress, performance trends,
    and knowledge sharing effectiveness across individual agents
    and multi-agent systems.
    """

    def __init__(self):
        print("📊 Learning visualizer initialized")

    def create_learning_dashboard(self, agent: AdaptiveAgent) -> Dict[str, Any]:
        """Create a comprehensive learning dashboard for an agent"""

        status = agent.get_agent_status()

        # Performance trends analysis
        performance_trend = self._analyze_performance_trend(agent.task_history)

        # Learning progress analysis
        learning_progress = self._analyze_learning_progress(agent)

        # Expertise development analysis
        expertise_growth = self._analyze_expertise_growth(agent)

        # Adaptation effectiveness analysis
        adaptation_analysis = self._analyze_adaptations(agent.adaptation_history)

        return {
            'agent_overview': {
                'name': agent.name,
                'tasks_completed': status['tasks_completed'],
                'current_success_rate': status['current_performance']['success_rate'],
                'exploration_rate': status['current_performance']['exploration_rate']
            },
            'performance_trends': performance_trend,
            'learning_progress': learning_progress,
            'expertise_development': expertise_growth,
            'adaptation_effectiveness': adaptation_analysis,
            'recommendations': self._generate_learning_recommendations(agent)
        }

    def _analyze_performance_trend(self, task_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Analyze performance trends over time"""

        if len(task_history) < 5:
            return {'message': 'Insufficient data for trend analysis', 'trend': 'insufficient_data'}

        # Group tasks into windows for trend analysis
        window_size = max(3, len(task_history) // 8)  # Adaptive window size
        windows = []

        for i in range(0, len(task_history), window_size):
            window = task_history[i:i+window_size]
            avg_score = sum(task['performance']['overall_score'] for task in window) / len(window)
            success_rate = sum(1 for task in window if task['performance']['success']) / len(window)

            windows.append({
                'window_start': i,
                'window_end': min(i + window_size, len(task_history)),
                'avg_score': avg_score,
                'success_rate': success_rate,
                'task_count': len(window)
            })

        # Calculate overall trend
        if len(windows) >= 2:
            early_avg = sum(w['avg_score'] for w in windows[:len(windows)//2]) / (len(windows)//2)
            late_avg = sum(w['avg_score'] for w in windows[len(windows)//2:]) / (len(windows) - len(windows)//2)

            if late_avg > early_avg + 0.1:
                trend = 'improving'
            elif late_avg < early_avg - 0.1:
                trend = 'declining'
            else:
                trend = 'stable'
        else:
            trend = 'insufficient_data'

        return {
            'overall_trend': trend,
            'windows': windows,
            'improvement_rate': (windows[-1]['avg_score'] - windows[0]['avg_score']) / len(windows) if len(windows) > 1 else 0.0,
            'total_improvement': windows[-1]['avg_score'] - windows[0]['avg_score'] if len(windows) > 1 else 0.0
        }

    def _analyze_learning_progress(self, agent: AdaptiveAgent) -> Dict[str, Any]:
        """Analyze learning progress and knowledge acquisition"""

        learning_stats = agent.learning_engine.get_learning_statistics()

        # Pattern discovery rate
        total_experiences = learning_stats['memory_statistics']['total_experiences']
        patterns_per_experience = (learning_stats['patterns_discovered'] / max(total_experiences, 1))

        # Learning efficiency - compare early vs recent performance
        if len(agent.task_history) >= 8:
            early_tasks = agent.task_history[:len(agent.task_history)//4]
            recent_tasks = agent.task_history[-len(agent.task_history)//4:]

            early_avg = sum(task['performance']['overall_score'] for task in early_tasks) / len(early_tasks)
            recent_avg = sum(task['performance']['overall_score'] for task in recent_tasks) / len(recent_tasks)
            learning_efficiency = (recent_avg - early_avg) / max(len(agent.task_history), 1)
        else:
            learning_efficiency = 0.0

        return {
            'patterns_discovered': learning_stats['patterns_discovered'],
            'pattern_discovery_rate': patterns_per_experience,
            'learning_efficiency': learning_efficiency,
            'memory_utilization': learning_stats['memory_statistics']['capacity_utilization'],
            'total_experiences': total_experiences,
            'knowledge_quality': learning_stats['memory_statistics']['success_rate']
        }

    def _analyze_expertise_growth(self, agent: AdaptiveAgent) -> Dict[str, Any]:
        """Analyze expertise development across domains"""

        expertise_areas = agent.expertise_areas

        if not expertise_areas:
            return {'message': 'No expertise areas developed yet'}

        # Top expertise areas
        top_areas = sorted(expertise_areas.items(), key=lambda x: x[1], reverse=True)[:5]

        # Expertise distribution
        total_expertise = sum(expertise_areas.values())
        if total_expertise > 0:
            expertise_distribution = {area: level/total_expertise for area, level in expertise_areas.items()}
        else:
            expertise_distribution = {}

        # Specialization vs generalization index
        expertise_values = list(expertise_areas.values())
        if expertise_values and len(expertise_values) > 1:
            max_expertise = max(expertise_values)
            avg_expertise = sum(expertise_values) / len(expertise_values)
            specialization_index = max_expertise - avg_expertise
        else:
            specialization_index = 0.0

        return {
            'top_expertise_areas': top_areas,
            'total_domains': len(expertise_areas),
            'expertise_distribution': expertise_distribution,
            'specialization_index': specialization_index,
            'highest_expertise': max(expertise_values) if expertise_values else 0.0,
            'average_expertise': sum(expertise_values) / len(expertise_values) if expertise_values else 0.0
        }

    def _analyze_adaptations(self, adaptation_history: List[Dict[str, Any]]) -> Dict[str, Any]:
        """Analyze effectiveness of behavioral adaptations"""

        if len(adaptation_history) < 2:
            return {'message': 'Insufficient adaptation data'}

        # Adaptation frequency analysis
        if adaptation_history:
            time_between_adaptations = []
            for i in range(1, len(adaptation_history)):
                time_diff = adaptation_history[i]['timestamp'] - adaptation_history[i-1]['timestamp']
                time_between_adaptations.append(time_diff)

            avg_adaptation_interval = sum(time_between_adaptations) / len(time_between_adaptations)
        else:
            avg_adaptation_interval = 0.0

        # Parameter evolution analysis
        exploration_evolution = [adapt['new_exploration_rate'] for adapt in adaptation_history]
        confidence_evolution = [adapt['new_confidence_threshold'] for adapt in adaptation_history]

        return {
            'total_adaptations': len(adaptation_history),
            'adaptation_frequency': avg_adaptation_interval,
            'exploration_rate_trend': self._calculate_trend(exploration_evolution),
            'confidence_threshold_trend': self._calculate_trend(confidence_evolution),
            'latest_adaptation': adaptation_history[-1] if adaptation_history else None
        }

    def _calculate_trend(self, values: List[float]) -> str:
        """Calculate trend direction for a series of values"""
        if len(values) < 2:
            return 'insufficient_data'

        start_avg = sum(values[:len(values)//2]) / (len(values)//2)
        end_avg = sum(values[len(values)//2:]) / (len(values) - len(values)//2)

        if end_avg > start_avg * 1.05:
            return 'increasing'
        elif end_avg < start_avg * 0.95:
            return 'decreasing'
        else:
            return 'stable'

    def _generate_learning_recommendations(self, agent: AdaptiveAgent) -> List[str]:
        """Generate recommendations for improving learning"""

        recommendations = []
        status = agent.get_agent_status()

        # Performance-based recommendations
        success_rate = status['current_performance']['success_rate']
        if success_rate < 0.6:
            recommendations.append("Consider increasing exploration rate to discover better strategies")
        elif success_rate > 0.8:
            recommendations.append("Success rate is high - consider reducing exploration to exploit current knowledge")

        # Expertise-based recommendations
        expertise_areas = status['expertise_areas']
        if len(expertise_areas) < 3:
            recommendations.append("Try tasks in new domains to develop broader expertise")

        max_expertise = max(expertise_areas.values()) if expertise_areas else 0.0
        if max_expertise < 0.5:
            recommendations.append("Focus on specific domains to develop deeper expertise")

        # Learning progress recommendations
        learning_stats = status['learning_progress']
        if learning_stats['patterns_discovered'] < 5:
            recommendations.append("Engage in more varied tasks to discover useful patterns")

        return recommendations

    def generate_learning_report(self, agent: AdaptiveAgent) -> str:
        """Generate a comprehensive learning report"""

        dashboard = self.create_learning_dashboard(agent)

        report = []
        report.append("=" * 60)
        report.append("ADAPTIVE AGENT LEARNING REPORT")
        report.append("=" * 60)
        report.append("")

        # Overview section
        overview = dashboard['agent_overview']
        report.append("📊 AGENT OVERVIEW")
        report.append(f"   Name: {overview['name']}")
        report.append(f"   Tasks Completed: {overview['tasks_completed']}")
        report.append(f"   Current Success Rate: {overview['current_success_rate']:.1%}")
        report.append(f"   Exploration Rate: {overview['exploration_rate']:.3f}")
        report.append("")

        # Performance trends section
        trends = dashboard['performance_trends']
        if 'overall_trend' in trends:
            report.append("📈 PERFORMANCE TRENDS")
            report.append(f"   Overall Trend: {trends['overall_trend'].replace('_', ' ').title()}")
            if 'improvement_rate' in trends:
                report.append(f"   Improvement Rate: {trends['improvement_rate']:.3f} per task window")
            if 'total_improvement' in trends:
                report.append(f"   Total Improvement: {trends['total_improvement']:+.3f}")
            report.append("")

        # Learning progress section
        progress = dashboard['learning_progress']
        if 'patterns_discovered' in progress:
            report.append("🎓 LEARNING PROGRESS")
            report.append(f"   Patterns Discovered: {progress['patterns_discovered']}")
            report.append(f"   Learning Efficiency: {progress['learning_efficiency']:.3f}")
            report.append(f"   Knowledge Quality: {progress['knowledge_quality']:.1%}")
            report.append(f"   Memory Utilization: {progress['memory_utilization']:.1%}")
            report.append("")

        # Expertise development section
        expertise = dashboard['expertise_development']
        if 'top_expertise_areas' in expertise:
            report.append("🏆 EXPERTISE DEVELOPMENT")
            report.append(f"   Total Domains: {expertise['total_domains']}")
            report.append(f"   Highest Expertise: {expertise['highest_expertise']:.3f}")
            report.append(f"   Specialization Index: {expertise['specialization_index']:.3f}")
            report.append("   Top Areas:")
            for area, level in expertise['top_expertise_areas'][:3]:
                report.append(f"     {area}: {level:.3f}")
            report.append("")

        # Adaptations section
        adaptations = dashboard['adaptation_effectiveness']
        if 'total_adaptations' in adaptations:
            report.append("🔄 ADAPTATION ANALYSIS")
            report.append(f"   Total Adaptations: {adaptations['total_adaptations']}")
            if 'exploration_rate_trend' in adaptations:
                report.append(f"   Exploration Trend: {adaptations['exploration_rate_trend'].replace('_', ' ').title()}")
            if 'confidence_threshold_trend' in adaptations:
                report.append(f"   Confidence Trend: {adaptations['confidence_threshold_trend'].replace('_', ' ').title()}")
            report.append("")

        # Recommendations section
        recommendations = dashboard['recommendations']
        if recommendations:
            report.append("💡 RECOMMENDATIONS")
            for i, rec in enumerate(recommendations, 1):
                report.append(f"   {i}. {rec}")
            report.append("")

        report.append("=" * 60)
        return "\n".join(report)

    def analyze_multi_agent_system(self, learning_system: MultiAgentLearningSystem) -> Dict[str, Any]:
        """Analyze the performance of a multi-agent learning system"""

        analytics = learning_system.get_system_analytics()

        # Agent performance comparison
        agent_comparison = []
        for agent_id, performance in analytics['agent_performance'].items():
            agent_comparison.append({
                'id': agent_id,
                'name': performance['name'],
                'tasks_completed': performance['tasks_completed'],
                'success_rate': performance['success_rate'],
                'contributions': performance['contributions'],
                'expertise_breadth': len(performance['top_expertise'])
            })

        # Sort by overall effectiveness (success rate + contributions)
        agent_comparison.sort(
            key=lambda x: x['success_rate'] * 0.7 + (x['contributions'] / max(analytics['total_agents'], 1)) * 0.3,
            reverse=True
        )

        # Knowledge sharing analysis
        knowledge_sharing_health = {
            'transfer_rate': analytics['knowledge_transfers'] / max(analytics['total_tasks_completed'], 1),
            'collaboration_rate': analytics['collaboration_events'] / max(analytics['total_tasks_completed'], 1),
            'sharing_effectiveness': analytics['knowledge_sharing_effectiveness'],
            'knowledge_density': analytics['shared_knowledge_items'] / max(analytics['total_agents'], 1)
        }

        # System expertise landscape
        total_expertise = 0
        expertise_distribution = defaultdict(float)

        for agent_info in learning_system.agents.values():
            agent = agent_info['agent']
            for domain, level in agent.expertise_areas.items():
                expertise_distribution[domain] += level
                total_expertise += level

        return {
            'system_overview': analytics,
            'agent_rankings': agent_comparison,
            'knowledge_sharing_health': knowledge_sharing_health,
            'expertise_landscape': dict(expertise_distribution),
            'total_system_expertise': total_expertise
        }

# =============================================================================
# COMPREHENSIVE TESTING AND DEMONSTRATION
# =============================================================================

print("🧪 Testing Learning Visualization System...")
print()

# Create and train an agent for detailed analysis
print("🏃 Training agent for comprehensive visualization demo...")

demo_agent = AdaptiveAgent(
    name="VizDemoBot",
    learning_objectives=[LearningObjective.ACCURACY, LearningObjective.EFFICIENCY, LearningObjective.QUALITY]
)

# Comprehensive training scenarios
training_scenarios = [
    {
        'description': 'Handle escalated customer complaint',
        'data': {'urgency': 'high', 'customer_type': 'premium', 'issue': 'billing_error'},
        'context': {'domain': 'customer_service', 'task_type': 'complaint_handling'}
    },
    {
        'description': 'Optimize database query performance',
        'data': {'complexity': 'high', 'table_size': 'large', 'join_count': 5},
        'context': {'domain': 'database_optimization', 'task_type': 'performance_tuning'}
    },
    {
        'description': 'Design responsive user interface',
        'data': {'platform': 'mobile', 'target': 'professionals', 'complexity': 'medium'},
        'context': {'domain': 'ui_design', 'task_type': 'responsive_design'}
    },
    {
        'description': 'Analyze quarterly financial data',
        'data': {'dataset': 'revenue_data', 'timeframe': 'Q4', 'metrics': ['growth', 'churn']},
        'context': {'domain': 'financial_analysis', 'task_type': 'quarterly_review'}
    },
    {
        'description': 'Debug production system issue',
        'data': {'system': 'microservices', 'error_type': 'timeout', 'impact': 'critical'},
        'context': {'domain': 'system_debugging', 'task_type': 'production_issue'}
    },
    {
        'description': 'Write technical documentation',
        'data': {'api_type': 'REST', 'complexity': 'medium', 'audience': 'developers'},
        'context': {'domain': 'technical_writing', 'task_type': 'api_documentation'}
    }
]

# Train the agent through multiple rounds
print(f"   Running {len(training_scenarios)} scenarios across 3 training rounds...")

for round_num in range(3):
    print(f"   Round {round_num + 1}...")
    for i, scenario in enumerate(training_scenarios):
        result = demo_agent.execute_task(
            scenario['description'],
            scenario['data'],
            scenario['context']
        )

        # Simulate feedback on some tasks
        if round_num > 0 and i % 2 == 0:  # Feedback on every other task after round 1
            feedback = {
                'satisfactory': result['performance']['success'],
                'metrics': {'expert_rating': random.uniform(0.6, 0.9)},
                'lesson': f"Round {round_num+1} feedback: {'Good approach' if result['performance']['success'] else 'Try different method'}"
            }
            demo_agent.learn_from_feedback(result['task_id'], feedback)

print(f"   ✅ Training complete: {len(demo_agent.task_history)} total tasks")
print()

# Create visualizer and generate comprehensive analysis
visualizer = LearningVisualizer()

print("📊 Generating comprehensive learning dashboard...")
dashboard = visualizer.create_learning_dashboard(demo_agent)

print()
print("📋 LEARNING DASHBOARD ANALYSIS")
print("=" * 40)

# Agent overview
overview = dashboard['agent_overview']
print(f"🤖 Agent: {overview['name']}")
print(f"   Tasks Completed: {overview['tasks_completed']}")
print(f"   Success Rate: {overview['current_success_rate']:.1%}")
print(f"   Exploration Rate: {overview['exploration_rate']:.3f}")

# Performance trends
trends = dashboard['performance_trends']
if trends.get('overall_trend') != 'insufficient_data':
    print(f"\n📈 Performance Trends:")
    print(f"   Overall Trend: {trends['overall_trend'].title()}")
    print(f"   Total Improvement: {trends.get('total_improvement', 0):+.3f}")
    print(f"   Improvement Rate: {trends.get('improvement_rate', 0):.3f} per window")

    # Show trend visualization
    windows = trends.get('windows', [])
    if len(windows) >= 3:
        print(f"   Progress Visualization:")
        for i, window in enumerate(windows):
            bar_length = int(window['avg_score'] * 20)
            bar = "█" * bar_length + "░" * (20 - bar_length)
            print(f"     Window {i+1}: {bar} {window['avg_score']:.3f}")

# Learning progress
progress = dashboard['learning_progress']
print(f"\n🎓 Learning Progress:")
print(f"   Patterns Discovered: {progress['patterns_discovered']}")
print(f"   Learning Efficiency: {progress['learning_efficiency']:.3f}")
print(f"   Knowledge Quality: {progress['knowledge_quality']:.1%}")
print(f"   Memory Utilization: {progress['memory_utilization']:.1%}")
print(f"   Total Experiences: {progress['total_experiences']}")

# Expertise development
expertise = dashboard['expertise_development']
if 'top_expertise_areas' in expertise:
    print(f"\n🏆 Expertise Development:")
    print(f"   Total Domains: {expertise['total_domains']}")
    print(f"   Specialization Index: {expertise['specialization_index']:.3f}")
    print(f"   Top Expertise Areas:")

    for i, (area, level) in enumerate(expertise['top_expertise_areas'][:5], 1):
        expertise_bar = "█" * int(level * 20) + "░" * (20 - int(level * 20))
        print(f"     {i}. {area}: {level:.3f} {expertise_bar}")

# Adaptations
adaptations = dashboard['adaptation_effectiveness']
if 'total_adaptations' in adaptations:
    print(f"\n🔄 Adaptation Analysis:")
    print(f"   Total Adaptations: {adaptations['total_adaptations']}")
    print(f"   Exploration Trend: {adaptations.get('exploration_rate_trend', 'N/A').title()}")
    print(f"   Confidence Trend: {adaptations.get('confidence_threshold_trend', 'N/A').title()}")

# Recommendations
recommendations = dashboard['recommendations']
if recommendations:
    print(f"\n💡 Learning Recommendations:")
    for i, rec in enumerate(recommendations, 1):
        print(f"   {i}. {rec}")

# Generate and display comprehensive report
print(f"\n📄 Generating comprehensive learning report...")
report = visualizer.generate_learning_report(demo_agent)

print(f"\n📖 LEARNING REPORT PREVIEW:")
print("-" * 50)
# Show key sections of the report
lines = report.split('\n')
current_section = ""
for line in lines:
    if line.startswith('📊') or line.startswith('📈') or line.startswith('🎓') or line.startswith('🏆'):
        current_section = line
        print(line)
    elif line.startswith('   ') and current_section:
        print(line)
    elif line.startswith('='):
        break

# Test multi-agent system analysis
print(f"\n🏫 Testing Multi-Agent System Analysis...")

# Create a multi-agent system for demonstration
multi_system = MultiAgentLearningSystem("Demo Learning System")

# Create agents with different specializations
specialist_agents = [
    AdaptiveAgent(name="CustomerServiceExpert", learning_objectives=[LearningObjective.QUALITY]),
    AdaptiveAgent(name="TechnicalExpert", learning_objectives=[LearningObjective.ACCURACY]),
    AdaptiveAgent(name="EfficiencyExpert", learning_objectives=[LearningObjective.EFFICIENCY])
]

# Add pre-existing expertise to make analysis more interesting
specialist_agents[0].expertise_areas['customer_service'] = 0.8
specialist_agents[0].expertise_areas['communication'] = 0.7
specialist_agents[1].expertise_areas['database_optimization'] = 0.9
specialist_agents[1].expertise_areas['system_debugging'] = 0.7
specialist_agents[2].expertise_areas['process_optimization'] = 0.6
specialist_agents[2].expertise_areas['efficiency_analysis'] = 0.8

# Add agents to system
for i, agent in enumerate(specialist_agents):
    specialization = ['customer_service', 'technical_systems', 'process_optimization'][i]
    multi_system.add_agent(agent, specialization)

# Run a quick learning session
quick_scenarios = training_scenarios[:3]  # Use first 3 scenarios
session_result = multi_system.coordinate_learning_session(quick_scenarios)

print(f"   Session completed: {session_result['scenarios_completed']} scenarios")
print(f"   Knowledge transfers: {session_result['knowledge_shared']}")

# Analyze the multi-agent system
system_analysis = visualizer.analyze_multi_agent_system(multi_system)

print(f"\n📊 MULTI-AGENT SYSTEM ANALYSIS:")
print(f"   Total Agents: {system_analysis['system_overview']['total_agents']}")
print(f"   Total Tasks: {system_analysis['system_overview']['total_tasks_completed']}")
print(f"   Knowledge Transfers: {system_analysis['system_overview']['knowledge_transfers']}")
print(f"   Total System Expertise: {system_analysis['total_system_expertise']:.2f}")

print(f"\n🏆 Agent Performance Rankings:")
for i, agent_data in enumerate(system_analysis['agent_rankings'], 1):
    ranking_icon = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
    print(f"   {ranking_icon} {agent_data['name']}")
    print(f"      Tasks: {agent_data['tasks_completed']}, Success: {agent_data['success_rate']:.1%}")
    print(f"      Contributions: {agent_data['contributions']}, Expertise Breadth: {agent_data['expertise_breadth']}")

print(f"\n🤝 Knowledge Sharing Health:")
sharing_health = system_analysis['knowledge_sharing_health']
print(f"   Transfer Rate: {sharing_health['transfer_rate']:.3f} transfers/task")
print(f"   Collaboration Rate: {sharing_health['collaboration_rate']:.3f} collaborations/task")
print(f"   Sharing Effectiveness: {sharing_health['sharing_effectiveness']:.3f}")
print(f"   Knowledge Density: {sharing_health['knowledge_density']:.3f} items/agent")

print(f"\n🗺️ System Expertise Landscape:")
expertise_landscape = system_analysis['expertise_landscape']
sorted_expertise = sorted(expertise_landscape.items(), key=lambda x: x[1], reverse=True)

for domain, total_expertise in sorted_expertise[:6]:
    expertise_percentage = (total_expertise / system_analysis['total_system_expertise']) * 100
    bar_length = int(expertise_percentage / 5)  # Scale bar to fit
    expertise_bar = "█" * bar_length + "░" * (20 - bar_length)
    print(f"   {domain:<25} {total_expertise:>5.2f} {expertise_bar} {expertise_percentage:>5.1f}%")

print()
print("🎉 Learning Visualization System Complete!")
print()
print("✅ TUTORIAL 12 PART 6 COMPLETE!")
print("=" * 45)
print()
print("📊 What We Built in Part 6:")
print("   • LearningVisualizer - Comprehensive analysis engine")
print("   • Performance trend analysis with visual progress bars")
print("   • Learning progress tracking with efficiency metrics")
print("   • Expertise development analysis with specialization indices")
print("   • Behavioral adaptation effectiveness analysis")
print("   • Intelligent learning recommendations generation")
print("   • Multi-agent system analysis with rankings")
print("   • Knowledge sharing health assessment")
print("   • Comprehensive human-readable report generation")
print("   • System-wide expertise landscape visualization")
print()
print("🌟 COMPLETE TUTORIAL 12 SUMMARY:")
print("   ✅ Part 1: Foundation learning framework")
print("   ✅ Part 2: Sophisticated experience memory system")
print("   ✅ Part 3: Multi-algorithm learning engine")
print("   ✅ Part 4: Adaptive agents with behavioral evolution")
print("   ✅ Part 5: Multi-agent collaborative learning")
print("   ✅ Part 6: Comprehensive visualization and analysis")
print()
print("🚀 Your learning agents can now:")
print("   • Learn from multiple types of experiences")
print("   • Adapt their behavior based on performance")
print("   • Share knowledge and learn from each other")
print("   • Provide detailed insights into their learning journey")
print("   • Generate comprehensive analysis and recommendations")
print()
print("🎯 Ready for Tutorial 13: Error Handling and Recovery!")
print("   Next we'll make these learning systems resilient and robust!")

Tutorial 12 Part 6: Learning Visualization and Analysis
Building comprehensive analysis and visualization tools...

🧪 Testing Learning Visualization System...

🏃 Training agent for comprehensive visualization demo...
🧠 Experience memory initialized (capacity: 1000)
🎓 Learning engine initialized with 3 objectives
🤖 Created adaptive agent: VizDemoBot
   Learning objectives: ['accuracy', 'efficiency', 'quality']
   Initial exploration rate: 0.1
   Initial confidence threshold: 0.6
   Running 6 scenarios across 3 training rounds...
   Round 1...
🎯 VizDemoBot executing: Handle escalated customer complaint...
   💡 Found 0 recommendations from past experience
     📝 Using standard approach (no strong recommendations or expertise)
   🎲 Chose action: standard_approach
   ✅ Result: True (score: 0.728)
   📝 Stored experience: Handle escalated customer complaint...
     📈 customer_service expertise: 0.00 → 0.07
🎯 VizDemoBot executing: Optimize database query performance...
   💡 Found 0 recommendat

In [18]:
# What We Learned & Common Errors and Solutions

print("Tutorial 12: Learning from Experience - Complete Summary")

print("=" * 60)

print("Comprehensive overview of adaptive learning systems")

print()

# =============================================================================
# WHAT WE LEARNED - COMPREHENSIVE OVERVIEW
# =============================================================================

print("📚 WHAT WE LEARNED:")

print("=" * 40)

print()

print("🏗️ 1. FOUNDATION LEARNING FRAMEWORK (Part 1)")

print("   Core Concepts Mastered:")

print("   • LearningExperience - Individual learning moments with context")

print("   • LearningPattern - Extracted knowledge that generalizes")

print("   • 5 Learning Types: Supervised, Unsupervised, Reinforcement, Imitation, Transfer")

print("   • 5 Experience Types: Success, Failure, Feedback, Observation, Exploration")

print("   • 5 Learning Objectives: Accuracy, Efficiency, Quality, Adaptability, Robustness")

print("   • Relevance calculation and age tracking for experiences")

print("   • Pattern confidence and success rate modeling")

print()

print("💾 2. SOPHISTICATED EXPERIENCE MEMORY (Part 2)")

print("   Memory Architecture Mastered:")

print("   • ExperienceMemory with intelligent indexing systems")

print("   • Type-based, context-based, and temporal indexing")

print("   • Similarity search with relevance scoring")

print("   • Automatic pattern discovery from successful experiences")

print("   • Memory capacity management with intelligent eviction")

print("   • Statistical analysis of memory utilization")

print("   • Context clustering for pattern extraction")

print()

print("🎓 3. MULTI-ALGORITHM LEARNING ENGINE (Part 3)")

print("   Learning Processing Mastered:")

print("   • 5 distinct learning algorithms with different strengths:")

print("     - Supervised: Input-output mapping from labeled examples")

print("     - Unsupervised: Pattern discovery through clustering")

print("     - Reinforcement: Q-learning with reward optimization")

print("     - Imitation: Behavior copying from observation")

print("     - Transfer: Knowledge adaptation across domains")

print("   • Performance baseline tracking and trend analysis")

print("   • Learning recommendations based on context relevance")

print("   • Feature extraction from diverse input types")

print("   • Automatic pattern discovery and consolidation")

print()

print("🤖 4. ADAPTIVE AGENTS WITH BEHAVIORAL EVOLUTION (Part 4)")

print("   Agent Intelligence Mastered:")

print("   • Dynamic action selection (exploration vs exploitation)")

print("   • Multi-dimensional performance evaluation:")

print("     - Accuracy, efficiency, quality, success metrics")

print("   • Domain expertise tracking and development")

print("   • Behavioral adaptation based on performance trends:")

print("     - Dynamic exploration rate adjustment")

print("     - Confidence threshold optimization")

print("   • External feedback integration for supervised learning")

print("   • Knowledge transfer between related domains")

print("   • Observation-based learning from other agents")

print("   • Comprehensive status tracking and self-awareness")

print()

print("🏫 5. MULTI-AGENT COLLABORATIVE LEARNING (Part 5)")

print("   Collective Intelligence Mastered:")

print("   • Coordinated learning sessions across agent teams")

print("   • Cross-agent observation and imitation learning")

print("   • Knowledge consolidation and sharing mechanisms")

print("   • Performance improvement tracking in group settings")

print("   • Direct knowledge transfer between agents")

print("   • Collaboration history and contribution tracking")

print("   • System-wide analytics and agent specialization")

print("   • Best performer identification and learning propagation")

print()

print("📊 6. COMPREHENSIVE VISUALIZATION AND ANALYSIS (Part 6)")

print("   Analysis and Insight Mastered:")

print("   • Performance trend analysis with time windows")

print("   • Learning progress tracking and efficiency measurement")

print("   • Expertise development and specialization analysis")

print("   • Behavioral adaptation effectiveness evaluation")

print("   • Intelligent learning recommendations generation")

print("   • Multi-agent system analysis and rankings")

print("   • Knowledge sharing health assessment")

print("   • Human-readable comprehensive report generation")

print("   • Comparative analysis across different learning styles")

print("   • System-wide expertise landscape visualization")

print()

# =============================================================================
# KEY ARCHITECTURAL INSIGHTS
# =============================================================================

print("🏛️ KEY ARCHITECTURAL INSIGHTS:")

print("=" * 40)

print()


print("🔄 Learning Lifecycle Understanding:")

print("   • Experience → Pattern → Knowledge → Action → Performance → Adaptation")

print("   • Continuous feedback loops between all components")

print("   • Memory consolidation from short-term to long-term patterns")

print("   • Multi-modal learning integration for robust knowledge")

print()

print("⚖️ Exploration vs Exploitation Balance:")

print("   • Dynamic exploration rate based on recent performance")

print("   • Higher exploration when performance is poor")

print("   • Lower exploration when performance is consistently good")

print("   • Context-dependent confidence thresholds")

print()

print("🎯 Expertise Development Patterns:")

print("   • Domain-specific knowledge accumulation")
print("   • Specialization vs generalization trade-offs")
print("   • Transfer learning effectiveness between related domains")
print("   • Expertise-based action selection improvements")
print()

print("🤝 Collaborative Learning Benefits:")
print("   • Faster learning through observation of successful agents")
print("   • Knowledge sharing reduces individual learning time")
print("   • Collective intelligence emerges from agent interactions")
print("   • Specialization allows for division of cognitive labor")
print()

# =============================================================================
# PRODUCTION IMPLEMENTATION INSIGHTS
# =============================================================================

print("🏭 PRODUCTION IMPLEMENTATION INSIGHTS:")
print("=" * 40)
print()

print("📈 Scalability Considerations:")
print("   • Memory systems need capacity limits and cleanup strategies")
print("   • Pattern discovery should be triggered periodically, not continuously")
print("   • Learning recommendations should be cached for performance")
print("   • Agent coordination requires efficient communication protocols")
print()

print("🔧 Performance Optimization:")
print("   • Index-based retrieval for large experience databases")
print("   • Batch processing for pattern discovery")
print("   • Lazy evaluation of learning recommendations")
print("   • Efficient serialization for agent state persistence")
print()

print("🛡️ Robustness Requirements:")
print("   • Graceful degradation when learning components fail")
print("   • Memory overflow protection with intelligent eviction")
print("   • Confidence-based decision making to avoid poor choices")
print("   • Multiple learning pathways for redundancy")
print()

# =============================================================================
# COMMON ERRORS AND SOLUTIONS
# =============================================================================

print("⚠️  COMMON ERRORS AND SOLUTIONS:")
print("=" * 40)
print()

print("1. 🐛 MEMORY LEAKS AND UNBOUNDED GROWTH")
print("   Problem: Learning systems accumulating unlimited experiences")
print("   Symptoms:")
print("     • Continuously increasing memory usage")
print("     • Slower performance over time")
print("     • Eventually running out of memory")
print("   Root Causes:")
print("     • No capacity limits on experience storage")
print("     • Missing cleanup of old/irrelevant experiences")
print("     • Retaining all patterns regardless of usefulness")
print("   Solutions:")
print("     ✅ Implement memory capacity limits with intelligent eviction")
print("     ✅ Use age-based and relevance-based cleanup strategies")
print("     ✅ Periodic pruning of low-confidence patterns")
print("     ✅ Background maintenance threads for cleanup")
print("   Code Example:")
print("     # Good: Capacity-limited memory")
print("     if len(self.experiences) > self.capacity:")
print("         self._evict_old_experiences()")
print()

print("2. 🐛 OVERFITTING TO RECENT EXPERIENCES")
print("   Problem: Agent relies too heavily on latest experiences")
print("   Symptoms:")
print("     • Poor performance when conditions change")
print("     • Ignoring valuable historical patterns")
print("     • Unstable behavior with high variance")
print("   Root Causes:")
print("     • Recent experiences weighted too heavily")
print("     • No temporal balance in pattern discovery")
print("     • Missing confidence decay for old patterns")
print("   Solutions:")
print("     ✅ Implement balanced temporal weighting")
print("     ✅ Use confidence decay for aging patterns")
print("     ✅ Ensemble multiple patterns for decisions")
print("     ✅ Maintain historical performance baselines")
print("   Code Example:")
print("     # Good: Balanced temporal weighting")
print("     weight = base_weight * math.exp(-age * decay_rate)")
print()

print("3. 🐛 EXPLORATION VS EXPLOITATION IMBALANCE")
print("   Problem: Too much exploration (inefficient) or too little (stagnation)")
print("   Symptoms:")
print("     • Consistently poor performance (over-exploration)")
print("     • Performance plateau (under-exploration)")
print("     • No adaptation to changing conditions")
print("   Root Causes:")
print("     • Fixed exploration rates regardless of performance")
print("     • No adaptation mechanism for exploration")
print("     • Missing context-dependent exploration strategies")
print("   Solutions:")
print("     ✅ Dynamic exploration rate based on recent performance")
print("     ✅ Higher exploration when performance is poor")
print("     ✅ Context-specific exploration strategies")
print("     ✅ Gradual exploration decay as expertise grows")
print("   Code Example:")
print("     # Good: Adaptive exploration")
print("     if recent_success_rate < 0.4:")
print("         self.exploration_rate = min(0.3, self.exploration_rate * 1.1)")
print()

print("4. 🐛 POOR KNOWLEDGE TRANSFER")
print("   Problem: Learned knowledge doesn't apply to new domains")
print("   Symptoms:")
print("     • No performance benefit from previous learning")
print("     • Starting from scratch in related domains")
print("     • Isolated expertise with no generalization")
print("   Root Causes:")
print("     • Too specific pattern extraction")
print("     • No abstraction mechanisms")
print("     • Missing domain relationship modeling")
print("   Solutions:")
print("     ✅ Abstract pattern extraction at multiple levels")
print("     ✅ Domain similarity assessment for transfer")
print("     ✅ Gradual transfer with validation")
print("     ✅ Transfer learning confidence tracking")
print("   Code Example:")
print("     # Good: Validated transfer learning")
print("     transfer_factor = domain_similarity * source_confidence")
print("     target_expertise = source_expertise * transfer_factor")
print()

print("5. 🐛 LEARNING PLATEAU AND STAGNATION")
print("   Problem: Agent stops improving after initial gains")
print("   Symptoms:")
print("     • Flat performance curves after initial learning")
print("     • No new pattern discovery")
print("     • Repetitive behavior patterns")
print("   Root Causes:")
print("     • Insufficient exploration in exploitation phase")
print("     • No curriculum learning progression")
print("     • Missing novelty detection mechanisms")
print("   Solutions:")
print("     ✅ Curriculum learning with progressive difficulty")
print("     ✅ Novelty-based exploration incentives")
print("     ✅ Regular parameter adaptation")
print("     ✅ Cross-domain learning exposure")
print("   Code Example:")
print("     # Good: Progressive difficulty curriculum")
print("     difficulty = base_difficulty + (task_count / 100) * 0.1")
print()

print("6. 🐛 INCONSISTENT FEEDBACK INTEGRATION")
print("   Problem: External feedback not properly incorporated")
print("   Symptoms:")
print("     • No improvement despite feedback")
print("     • Conflicting learned patterns")
print("     • Ignoring expert corrections")
print("   Root Causes:")
print("     • Feedback treated same as other experiences")
print("     • No feedback quality assessment")
print("     • Missing feedback-specific learning paths")
print("   Solutions:")
print("     ✅ Separate feedback processing pipeline")
print("     ✅ Feedback quality and source assessment")
print("     ✅ Higher weight for validated feedback")
print("     ✅ Feedback contradiction detection")
print("   Code Example:")
print("     # Good: Weighted feedback processing")
print("     if experience.experience_type == ExperienceType.FEEDBACK:")
print("         weight *= feedback_importance_multiplier")
print()

print("7. 🐛 COORDINATION CONFLICTS IN MULTI-AGENT SYSTEMS")
print("   Problem: Agents interfering with each other's learning")
print("   Symptoms:")
print("     • Degraded performance in group settings")
print("     • Conflicting recommendations")
print("     • Knowledge sharing failures")
print("   Root Causes:")
print("     • No coordination protocols")
print("     • Conflicting learning objectives")
print("     • Resource competition between agents")
print("   Solutions:")
print("     ✅ Clear coordination protocols and roles")
print("     ✅ Conflict resolution mechanisms")
print("     ✅ Shared knowledge validation")
print("     ✅ Performance-based coordination adjustment")
print("   Code Example:")
print("     # Good: Conflict-aware coordination")
print("     if recommendation_confidence > threshold:")
print("         shared_knowledge[pattern_id] = pattern")
print()

print("8. 🐛 MEMORY RETRIEVAL PERFORMANCE DEGRADATION")
print("   Problem: Learning system becomes slow with accumulated data")
print("   Symptoms:")
print("     • Increasing response times over time")
print("     • High CPU usage during retrieval")
print("     • Memory search timeouts")
print("   Root Causes:")
print("     • Linear search through large memory stores")
print("     • No indexing for fast retrieval")
print("     • Inefficient similarity calculations")
print("   Solutions:")
print("     ✅ Multi-level indexing systems")
print("     ✅ Approximate similarity search for speed")
print("     ✅ Caching of frequent retrieval results")
print("     ✅ Background index optimization")
print("   Code Example:")
print("     # Good: Indexed retrieval")
print("     candidates = self.context_index[context_key]")
print("     # Instead of searching all experiences")
print()

print("9. 🐛 PATTERN DISCOVERY EXPLOSION")
print("   Problem: Too many weak patterns cluttering the system")
print("   Symptoms:")
print("     • Thousands of low-confidence patterns")
print("     • Slow pattern matching")
print("     • Difficulty finding relevant patterns")
print("   Root Causes:")
print("     • No minimum confidence thresholds")
print("     • Pattern creation from insufficient data")
print("     • No pattern pruning mechanisms")
print("   Solutions:")
print("     ✅ Minimum observation thresholds for pattern creation")
print("     ✅ Confidence-based pattern pruning")
print("     ✅ Pattern consolidation and merging")
print("     ✅ Periodic pattern quality assessment")
print("   Code Example:")
print("     # Good: Quality-gated pattern creation")
print("     if len(group_experiences) >= min_observations:")
print("         if pattern_confidence > min_confidence:")
print("             create_pattern(group_experiences)")
print()

print("10. 🐛 CONTEXT MISMATCH IN LEARNING")
print("    Problem: Learning applied in inappropriate contexts")
print("    Symptoms:")
print("      • Poor performance despite extensive learning")
print("      • Inappropriate action selection")
print("      • Context-sensitive failures")
print("    Root Causes:")
print("      • Insufficient context representation")
print("      • Poor context similarity metrics")
print("      • Missing context validation")
print("    Solutions:")
print("      ✅ Rich context representation with multiple dimensions")
print("      ✅ Sophisticated context similarity measures")
print("      ✅ Context-specific confidence adjustments")
print("      ✅ Context validation before pattern application")
print("    Code Example:")
print("      # Good: Context-aware pattern matching")
print("      relevance = self._calculate_pattern_relevance(pattern, context)")
print("      if relevance > min_relevance_threshold:")
print("          apply_pattern(pattern)")
print()

# =============================================================================
# DEBUGGING AND TROUBLESHOOTING GUIDE
# =============================================================================

print("🔧 DEBUGGING AND TROUBLESHOOTING GUIDE:")
print("=" * 40)
print()

print("🔍 Performance Issues:")
print("   Symptom: Slow learning or poor adaptation")
print("   Debug Steps:")
print("     1. Check memory utilization and pattern count")
print("     2. Analyze exploration rate and adaptation frequency")
print("     3. Examine pattern confidence distributions")
print("     4. Review context similarity calculations")
print("   Diagnostic Code:")
print("     stats = agent.get_agent_status()")
print("     print(f'Patterns: {stats[\"learning_progress\"][\"patterns_discovered\"]}')")
print("     print(f'Memory: {stats[\"learning_progress\"][\"memory_utilization\"]}')")
print()

print("🔍 Learning Stagnation:")
print("   Symptom: No improvement in performance over time")
print("   Debug Steps:")
print("     1. Check if exploration rate is too low")
print("     2. Verify pattern discovery is occurring")
print("     3. Examine adaptation trigger frequency")
print("     4. Review task diversity and difficulty progression")
print("   Diagnostic Code:")
print("     trends = visualizer._analyze_performance_trend(agent.task_history)")
print("     print(f'Trend: {trends[\"overall_trend\"]}')")
print("     print(f'Improvement: {trends[\"improvement_rate\"]}')")
print()

print("🔍 Memory Problems:")
print("   Symptom: High memory usage or slow retrieval")
print("   Debug Steps:")
print("     1. Check experience memory capacity utilization")
print("     2. Verify eviction policies are working")
print("     3. Examine pattern pruning effectiveness")
print("     4. Review indexing performance")
print("   Diagnostic Code:")
print("     memory_stats = agent.learning_engine.experience_memory.get_statistics()")
print("     print(f'Capacity: {memory_stats[\"capacity_utilization\"]}')")
print("     print(f'Patterns: {memory_stats[\"patterns_discovered\"]}')")
print()

# =============================================================================
# BEST PRACTICES SUMMARY
# =============================================================================

print("🌟 BEST PRACTICES SUMMARY:")
print("=" * 40)
print()

print("🎯 Design Principles:")
print("   • Start with simple learning objectives and expand gradually")
print("   • Implement comprehensive logging and monitoring from the beginning")
print("   • Design for adaptability - parameters should be tunable")
print("   • Build in multiple learning pathways for robustness")
print("   • Plan for scalability with proper indexing and cleanup")
print()

print("🔄 Implementation Strategy:")
print("   • Implement parts sequentially as shown in tutorial structure")
print("   • Test each component thoroughly before integration")
print("   • Use realistic scenarios for testing and validation")
print("   • Monitor performance and adaptation effectiveness continuously")
print("   • Implement graceful degradation for component failures")
print()

print("📊 Monitoring and Maintenance:")
print("   • Track learning progress and adaptation patterns")
print("   • Monitor memory usage and retrieval performance")
print("   • Analyze knowledge sharing effectiveness in multi-agent systems")
print("   • Regular pattern quality assessment and cleanup")
print("   • Performance trend analysis for early problem detection")
print()

print("🚀 Ready for Tutorial 13: Error Handling and Recovery!")
print("   Next we'll make these learning systems resilient and fault-tolerant!")
print()

# =============================================================================
# IMPLEMENTATION CHECKLIST
# =============================================================================

print("✅ IMPLEMENTATION CHECKLIST:")
print("=" * 40)
print()

implementation_checklist = [
    "Foundation Classes (LearningExperience, LearningPattern)",
    "Experience Memory with indexing and capacity management",
    "Learning Engine with multiple algorithms",
    "Adaptive Agent with behavioral evolution",
    "Multi-Agent Learning System with coordination",
    "Visualization and Analysis tools",
    "Memory cleanup and eviction strategies",
    "Performance monitoring and trend analysis",
    "Context-aware pattern matching",
    "Dynamic exploration rate adaptation",
    "Knowledge transfer validation",
    "Feedback integration pipeline",
    "Error handling and graceful degradation",
    "Scalability considerations and optimization",
    "Comprehensive testing with realistic scenarios"
]

for i, item in enumerate(implementation_checklist, 1):
    print(f"   {i:2d}. {item}")

print()

print("🎉 Tutorial 12 Complete - You now have comprehensive")

print("   adaptive learning systems that can continuously improve!")

Tutorial 12: Learning from Experience - Complete Summary
Comprehensive overview of adaptive learning systems

📚 WHAT WE LEARNED:

🏗️ 1. FOUNDATION LEARNING FRAMEWORK (Part 1)
   Core Concepts Mastered:
   • LearningExperience - Individual learning moments with context
   • LearningPattern - Extracted knowledge that generalizes
   • 5 Learning Types: Supervised, Unsupervised, Reinforcement, Imitation, Transfer
   • 5 Experience Types: Success, Failure, Feedback, Observation, Exploration
   • 5 Learning Objectives: Accuracy, Efficiency, Quality, Adaptability, Robustness
   • Relevance calculation and age tracking for experiences
   • Pattern confidence and success rate modeling

💾 2. SOPHISTICATED EXPERIENCE MEMORY (Part 2)
   Memory Architecture Mastered:
   • ExperienceMemory with intelligent indexing systems
   • Type-based, context-based, and temporal indexing
   • Similarity search with relevance scoring
   • Automatic pattern discovery from successful experiences
   • Memory capaci